In [4]:
#Script para gerar Tabela Produto e enviar direto para a tabela no banco de dados

import pandas as pd
import random
from datetime import date
from sqlalchemy import create_engine, VARCHAR,NUMERIC,INTEGER,DATE,DATETIME,String,text,CHAR
import numpy as np
from faker import Faker

quantidade_produtos_gerados = 250000
tabela = "PRODUTO" #NOME DA TABELA A SEREM ADICIONADOS OS DADOS
coluna = "NOME"    #NOME DA COLUNA DA TABELA QUE O SCRCIPT FARA A VERIFICACAO SE O DADO JA EXISTE NO BANCO DE DADOS

server = "DESKTOP-HV85ADA\\PROJETO_MERCADO"
bancodados = "MERCADO_OLTP"
driver = "ODBC Driver 17 for SQL Server"
#user = ""
#senha = ""

conexaostring = f"mssql+pyodbc://{server}/{bancodados}?driver={driver}&Trusted_Connection=yes"
#conn_str = f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver={driver}"
engine = create_engine(conexaostring)

#Função para testar a conexão
def testar_conexao_sqlalchemy(engine):
    try:        
        with engine.connect() as conexao: #Tenta conectar ao banco de dados
            print("Conexão com o banco de dados estabelecida com sucesso")
            result = conexao.execute(text("SELECT 1 AS teste")) #Executa uma consulta para testar a conexão
            print("Query executada com sucesso")
            return True
    except Exception as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return False
#testa a conexão
if testar_conexao_sqlalchemy(engine): 
    print("A conexão foi bem-sucedida")
else:
    print("Falha na conexão.")

produtos = [
    "Arroz", "Feijao", "Acucar", "Sal", "Oleo de soja", "Cafe", "Farinha de trigo", "Leite", "Ovos", "Pao", "Macarrao", "Molho de tomate", "Carne bovina", "Frango", "Peixe", "Salsicha", "Presunto", "Queijo", "Manteiga", "Margarina",
    "Iogurte", "Requeijao", "Biscoito", "Bolacha", "Sabonete", "Shampoo", "Condicionador", "Papel higienico", "Detergente", "Amaciante", "Agua sanitaria", "Desinfetante", "Esponja de lavar louca", "Sacola plastica", "Guardanapo", 
    "Toalha de papel", "Filtro de cafe", "Cereal matinal", "Achocolatado", "Refrigerante", "Suco", "Cerveja", "Vinho", "Agua mineral", "Cha", "Salgadinho", "Chocolate", "Balas", "Goma de mascar", "Sorvete","Milho", "Ervilha", 
    "Seleta de legumes", "Extrato de tomate", "Maionese", "Mostarda", "Ketchup", "Tempero pronto", "Caldo de carne", "Caldo de galinha", "Farinha de mandioca", "Polenta", "Lentilha", "Grao-de-bico", "Soja", "Aveia", "Granola", 
    "Barra de cereal", "Pipoca", "Amendoim", "Castanha", "Nozes", "Uva passa", "Azeite", "Vinagre", "Fermento em po", "Gelatina", "Pudim", "Leite condensado", "Creme de leite", "Chantilly", "Coco ralado", "Canela", "Fermento biologico", 
    "Linguica", "Carne moida", "Costela", "Bacon", "Hamburguer", "Lasanha", "Pizza", "Pao de queijo", "Tapioca", "Geleia", "Mel", "Pasta de amendoim", "Sardinha", "Atum", "Sopa instantanea", "Batata", "Cebola", "Tomate", 
    "Alho", "Cenoura", "Abobora", "Chuchu", "Pimentao", "Alface", "Repolho","Couve", "Brocolis", "Banana", "Maca", "Laranja", "Limao", "Mamao", "Abacaxi", "Melancia", "Melao", "Uva", "Morango", "Kiwi", "Pera", "Manga", "Goiaba", "Caju", 
    "Maracuja", "Acerola", "Coco", "Pessego", "Ameixa", "Figo", "Caqui", "Jabuticaba", "Cupuacu", "Acai", "Guarana", "Pinhao", "Palmito", "Azeitona", "Pepino", "Rabanete", "Beterraba", "Inhame", "Mandioca", "Batata-doce", "Gengibre", 
    "Coentro", "Salsinha", "Farinha de rosca", "Fermento quimico", "Cacau em po", "Cha mate", "Cha verde", "Cha de camomila", "Cha de hortela", "Cafe soluvel", "Cafe em capsula", "Cafe descafeinado", "Leite em po", "Leite de soja", 
    "Leite de amendoas", "Leite de coco", "Leite de aveia", "Leite sem lactose", "Iogurte grego", "Iogurte natural", "Iogurte com frutas", "Iogurte zero lactose", "Queijo minas", "Queijo prato", "Queijo mussarela", "Queijo parmesao", 
    "Queijo provolone", "Queijo cottage", "Queijo ricota", "Queijo gorgonzola", "Queijo brie", "Queijo coalho", "Presunto cozido", "Presunto defumado", "Peito de peru", "Salame", "Mortadela", "Rosbife", "Carne seca", "Charque", 
    "Bacalhau", "Tilapia", "Salmao", "Camarao", "Lula", "Polvo", "Ostra", "Mexilhao", "Sushi", "Temaki", "Yakult", "Danoninho", "Biscoito recheado", "Biscoito agua e sal", "Biscoito cream cracker", "Biscoito de polvilho", "Biscoito de aveia",
    "Biscoito de gengibre", "Biscoito de chocolate", "Biscoito de coco", "Biscoito de maisena", "Biscoito de nata", "Pao integral", "Pao frances", "Pao de forma", "Pao sirio", "Pao de centeio", "Pao de aveia", "Pao de milho", "Pao de batata", 
    "Pao de queijo congelado", "Pao de alho", "Salsicha hot dog", "Salsicha para churrasco", "Salsicha de frango", "Salsicha de peru", "Salsicha vegetariana", "Hamburguer de frango", "Hamburguer vegetariano", "Hamburguer de soja", 
    "Hamburguer de peixe", "Hamburguer artesanal", "Pipoca doce", "Pipoca salgada", "Pipoca de micro-ondas", "Pipoca caramelizada", "Pipoca com manteiga", "Pipoca com queijo", "Pipoca com bacon", "Pipoca colorida", "Pipoca zero acucar", 
    "Pipoca zero gordura","Pizza congelada", "Lasanha congelada", "Pao de queijo congelado", "Hamburguer congelado", "Batata frita congelada", "Nuggets de frango", "Empanado de frango", "Empanado de peixe", "Escondidinho congelado", "Feijoada congelada",
    "Risoto congelado", "Yakisoba congelado", "Coxinha congelada", "Kibe congelado", "Pastel congelado", "Torta congelada", "Pao de alho congelado", "Pao de queijo recheado congelado", "Sorvete de massa", "Sorvete de palito",
    "Acai congelado", "Polenta congelada", "Legumes congelados", "Frutas congeladas", "Pure de batata congelado", "Pao frances congelado", "Pao de forma congelado", "Pao integral congelado", "Pao sirio congelado", "Pao de centeio congelado",
    "Vodka", "Whisky", "Rum", "Gin", "Tequila", "Cachaca", "Licor", "Champagne", "Espumante", "Sidra", "Conhaque", "Vermute", "Sake", "Absinto", "Catuaba", "Pinga artesanal", "Vinho tinto", "Vinho branco", "Vinho rose", "Vinho seco", 
    "Vinho suave", "Cerveja artesanal", "Cerveja sem alcool", "Cerveja de trigo", "Cerveja IPA", "Cerveja stout", "Cerveja pilsen", "Cerveja light", "Cerveja zero alcool", "Cerveja de milho", 
    "Oleo de girassol", "Oleo de milho", "Oleo de canola", "Oleo de coco", "Oleo de linhaca", "Oleo de gergelim", "Oleo de amendoim", "Oleo de algodao", "Oleo de palma", "Oleo de abacate",
    "Arroz integral", "Arroz arboreo", "Arroz negro", "Arroz vermelho", "Arroz cateto", "Arroz jasmine", "Arroz basmati", "Arroz parboilizado", "Arroz selvagem", "Arroz 7 graos",
    "Farinha de milho", "Farinha de aveia", "Farinha de centeio", "Farinha de cevada", "Farinha de coco", "Farinha de amendoas", "Farinha de arroz", "Farinha de grao-de-bico", "Farinha de quinoa", "Farinha de linhaca",
    "Farinha de tapioca", "Farinha de banana verde", "Farinha de maracuja", "Farinha de berinjela", "Farinha de soja", "Farinha de trigo integral", "Farinha de trigo sarraceno", "Farinha de trigo sem gluten", "Farinha de trigo especial", "Farinha de trigo tipo 1",
    "Desodorante roll-on", "Desodorante aerosol", "Sabonete liquido", "Sabonete em barra", "Creme para barbear", "Locao pos-barba", "Protetor solar", "Hidratante corporal", "Lenco umedecido", "Absorvente noturno",
    "Absorvente diario", "Cotonete", "Fio dental", "Enxaguante bucal", "Pasta de dente clareadora", "Escova de dentes infantil", "Escova de dentes eletrica", "Creme para pentear", "Gel para cabelo", "Mascara capilar",
    "Condicionador reparador", "Shampoo anticaspa", "Shampoo infantil", "Sabonete intimo", "Creme para assaduras", "Oleo para bebe", "Lenco de papel", "Papel higienico perfumado", "Papel higienico biodegradavel", "Sabonete antibacteriano",
    "Desinfetante para as maos", "Creme para os pes", "Esmalte de unha", "Removedor de esmalte", "Cera depilatoria", "Lamina de barbear descartavel", "Creme para rugas", "Protetor labial", "Po compacto", "Batom",
    "Base liquida", "Demaquilante", "Creme para olheiras", "Creme para acne", "Protetor solar facial", "Shampoo seco", "Condicionador sem enxague", "Gel para sobrancelha", "Creme para maos", "Sabonete esfoliante",
    "Detergente neutro", "Detergente concentrado", "Amaciante perfumado", "Amaciante hipoalergenico", "Desinfetante para banheiro", "Desinfetante para cozinha", "Limpador multiuso", "Limpador para vidros", "Limpador para pisos", "Lustra-moveis",
    "Saco de lixo", "Saco de lixo reciclavel", "Saco de lixo biodegradavel", "Vassoura", "Rodo", "Pa de lixo", "Balde", "Esponja de aco", "Esponja multiuso", "Luvas de limpeza",
    "Alcool em gel", "Alcool liquido", "Limpa-vidros", "Limpa-forno", "Limpa-carpete", "Limpa-azulejo", "Limpa-inox", "Limpa-pisos", "Limpa-tapetes", "Limpa-estofados",
    "Desengordurante", "Removedor de manchas", "Cera para pisos", "Cera para moveis", "Cera para carro", "Inseticida", "Repelente de insetos", "Aromatizador de ambientes", "Difusor de aromas", "Velas aromaticas",
    "Panela de pressao", "Panela antiaderente", "Panela de ferro", "Panela de barro", "Frigideira", "Chaleira", "Liquidificador", "Batedeira", "Processador de alimentos", "Torradeira",
    "Forno eletrico", "Air fryer", "Cafeteira", "Jarra eletrica", "Faca de cozinha", "Tabua de corte", "Escorredor de louca", "Espatula de silicone", "Colher de pau", "Ralador de queijo",
    "Descascador de legumes", "Assadeira", "Forma de bolo", "Forma de gelo", "Garrafa termica", "Caneca", "Prato", "Tigela", "Talheres", "Guardanapo de pano",
    "Toalha de mesa", "Jogo americano", "Vaso decorativo", "Porta-temperos", "Porta-sabao", "Escova de limpeza", "Rodo de pia", "Vassoura de pia", "Organizador de gavetas", "Cabide",
    "Caderno", "Caneta", "Lapis", "Borracha", "Apontador", "Regua", "Estojo", "Mochila", "Cola branca", "Tesoura"
]

mercearia = [
    "Arroz", "Feijao", "Acucar", "Sal", "Oleo de soja", "Cafe", "Farinha de trigo", "Macarrao", "Molho de tomate", "Farinha de mandioca", "Polenta", "Lentilha", "Grao-de-bico", "Soja", "Aveia", "Granola", "Barra de cereal", "Pipoca", "Amendoim", "Castanha",
    "Nozes", "Uva passa", "Azeite", "Vinagre", "Fermento em po", "Gelatina", "Pudim", "Leite condensado", "Creme de leite", "Chantilly","Coco ralado", "Canela", "Fermento biologico", "Tapioca", "Geleia", "Mel", "Pasta de amendoim", "Sopa instantanea", 
    "Filtro de cafe", "Cereal matinal", "Achocolatado", "Extrato de tomate", "Maionese", "Mostarda", "Ketchup", "Tempero pronto", "Caldo de carne", "Caldo de galinha", "Seleta de legumes", "Farinha de rosca",
    "Fermento quimico", "Cacau em po", "Cha mate", "Cha verde", "Cha de camomila", "Cha de hortela", "Cafe soluvel", "Cafe em capsula", "Cafe descafeinado", "Oleo de girassol",
    "Oleo de milho", "Oleo de canola", "Oleo de coco", "Oleo de linhaca", "Oleo de gergelim", "Oleo de amendoim", "Oleo de algodao", "Oleo de palma", "Oleo de abacate", "Arroz integral",
    "Arroz arboreo", "Arroz negro", "Arroz vermelho", "Arroz cateto", "Arroz jasmine", "Arroz basmati", "Arroz parboilizado", "Arroz selvagem", "Arroz 7 graos", "Farinha de milho",
    "Farinha de aveia", "Farinha de centeio", "Farinha de cevada", "Farinha de coco", "Farinha de amendoas", "Farinha de arroz", "Farinha de grao-de-bico", "Farinha de quinoa", "Farinha de linhaca", "Farinha de tapioca",
    "Farinha de banana verde", "Farinha de maracuja", "Farinha de berinjela", "Farinha de soja", "Farinha de trigo integral", "Farinha de trigo sarraceno", "Farinha de trigo sem gluten", "Farinha de trigo especial", "Farinha de trigo tipo 1",
    "Salgadinho", "Chocolate", "Balas", "Goma de mascar", "Pipoca doce", "Pipoca salgada", "Pipoca de micro-ondas", "Pipoca caramelizada", "Pipoca com manteiga", "Pipoca com queijo","Pipoca com bacon", "Pipoca colorida", "Pipoca zero acucar", 
    "Pipoca zero gordura", "Sopa instantanea", "Farinha de trigo tipo 1", "Filtro de cafe", "Cereal matinal", "Achocolatado", "Leite em po", "Leite de soja", "Leite de amendoas", "Leite de coco", "Leite de aveia", 
    "Leite sem lactose", "Biscoito recheado", "Biscoito agua e sal", "Biscoito cream cracker", "Biscoito de polvilho", "Biscoito de aveia", "Biscoito de gengibre", "Biscoito de chocolate", "Biscoito de coco", "Biscoito de maisena", "Biscoito de nata"
]

produtos_limpeza = [
    "Detergente", "Amaciante", "Agua sanitaria", "Desinfetante", "Esponja de lavar louca", "Sacola plastica", "Guardanapo", "Toalha de papel", "Sabao em po", "Limpador multiuso",
    "Detergente neutro", "Detergente concentrado", "Amaciante perfumado", "Amaciante hipoalergenico", "Desinfetante para banheiro", "Desinfetante para cozinha", "Limpador multiuso", "Limpador para vidros", "Limpador para pisos", "Lustra-moveis",
    "Saco de lixo", "Saco de lixo reciclavel", "Saco de lixo biodegradavel", "Vassoura", "Rodo", "Pa de lixo", "Balde", "Esponja de aco", "Esponja multiuso", "Luvas de limpeza",
    "Alcool em gel", "Alcool liquido", "Limpa-vidros", "Limpa-forno", "Limpa-carpete", "Limpa-azulejo", "Limpa-inox", "Limpa-pisos", "Limpa-tapetes", "Limpa-estofados",
    "Desengordurante", "Removedor de manchas", "Cera para pisos", "Cera para moveis", "Cera para carro", "Inseticida", "Repelente de insetos", "Aromatizador de ambientes", "Difusor de aromas", "Velas aromaticas"
]

higiene_pessoal = [
    "Sabonete", "Shampoo", "Condicionador", "Papel higienico", "Creme dental", "Escova de dentes", "Desodorante", "Absorvente", "Lamina de barbear", "Creme hidratante",
    "Desodorante roll-on", "Desodorante aerosol", "Sabonete liquido", "Sabonete em barra", "Creme para barbear", "Locao pos-barba", "Protetor solar", "Hidratante corporal", "Lenco umedecido", "Absorvente noturno",
    "Absorvente diario", "Cotonete", "Fio dental", "Enxaguante bucal", "Pasta de dente clareadora", "Escova de dentes infantil", "Escova de dentes eletrica", "Creme para pentear", "Gel para cabelo", "Mascara capilar",
    "Condicionador reparador", "Shampoo anticaspa", "Shampoo infantil", "Sabonete intimo", "Creme para assaduras", "Oleo para bebe", "Lenco de papel", "Papel higienico perfumado", "Papel higienico biodegradavel", "Sabonete antibacteriano",
    "Desinfetante para as maos", "Creme para os pes", "Esmalte de unha", "Removedor de esmalte", "Cera depilatoria", "Lamina de barbear descartavel", "Creme para rugas", "Protetor labial", "Po compacto", "Batom",
    "Base liquida", "Demaquilante", "Creme para olheiras", "Creme para acne", "Protetor solar facial", "Shampoo seco", "Condicionador sem enxague", "Gel para sobrancelha", "Creme para maos", "Sabonete esfoliante"
]

bebidas_nao_alcoolicas = [
    "Refrigerante", "Suco", "Agua mineral", "Cha", "Achocolatado", "Cafe", "Leite", "Energetico", "Cerveja sem alcool", "Cerveja zero alcool"
]

bebidas_alcoolicas = [
    "Cerveja", "Vinho", "Vodka", "Whisky", "Rum", "Gin", "Tequila", "Cachaca", "Licor", "Champagne",
    "Espumante", "Sidra", "Conhaque", "Vermute", "Sake", "Absinto", "Catuaba", "Pinga artesanal", "Vinho tinto", "Vinho branco",
    "Vinho rose", "Vinho seco", "Vinho suave", "Cerveja artesanal", "Cerveja de trigo", "Cerveja IPA", "Cerveja stout", "Cerveja pilsen", "Cerveja light", "Cerveja de milho"
]

laticinios = [
    "Leite", "Queijo", "Manteiga", "Margarina", "Iogurte", "Requeijao", "Creme de leite", "Leite condensado", "Chantilly", "Fermento biologico", "Leite em po", "Leite de soja", "Leite de amendoas", "Leite de coco","Leite de aveia", "Leite sem lactose", 
    "Iogurte grego", "Iogurte natural", "Iogurte com frutas", "Iogurte zero lactose", "Queijo minas", "Queijo prato", "Queijo mussarela", "Queijo parmesao", "Queijo provolone", "Queijo cottage", "Queijo ricota", "Queijo gorgonzola", "Queijo brie", 
    "Queijo coalho", "Yakult", "Danoninho", "Iogurte grego", "Iogurte natural", "Iogurte com frutas", "Iogurte zero lactose", "Queijo minas", "Queijo prato", "Queijo mussarela", "Queijo parmesao", "Queijo provolone", "Queijo cottage", "Queijo ricota", "Queijo gorgonzola", 
    "Queijo brie", "Queijo coalho"
]

carnes = [
    "Carne bovina", "Frango", "Peixe", "Salsicha", "Presunto", "Linguica", "Carne moida", "Costela", "Bacon", "Hamburguer", "Presunto cozido", "Presunto defumado", "Peito de peru", "Salame", "Mortadela", "Rosbife", "Carne seca", "Charque", "Bacalhau", "Tilapia",
    "Salmao", "Camarao", "Lula", "Polvo", "Ostra", "Mexilhao", "Sushi", "Temaki", "Salsicha hot dog", "Salsicha para churrasco", "Salsicha de frango", "Salsicha de peru", "Salsicha vegetariana", "Hamburguer de frango", "Hamburguer vegetariano",
    "Hamburguer de soja", "Hamburguer de peixe", "Hamburguer artesanal", "Presunto cozido", "Presunto defumado", "Peito de peru", "Salame", "Mortadela", "Rosbife", "Carne seca", "Charque", "Bacalhau", "Tilapia", "Salmao", "Camarao", "Lula", "Polvo", 
    "Ostra", "Mexilhao", "Sushi", "Temaki", "Salsicha hot dog", "Salsicha para churrasco", "Salsicha de frango", "Salsicha de peru", "Salsicha vegetariana", "Hamburguer de frango", "Hamburguer vegetariano", "Hamburguer de soja", "Hamburguer de peixe", 
    "Hamburguer artesanal"
]

congelados = [
    "Pizza congelada", "Lasanha congelada", "Pao de queijo congelado", "Hamburguer congelado", "Batata frita congelada", "Nuggets de frango", "Empanado de frango", "Empanado de peixe", "Escondidinho congelado", "Feijoada congelada", 
    "Pao de queijo congelado", "Risoto congelado", "Yakisoba congelado", "Coxinha congelada", "Kibe congelado", "Pastel congelado", "Torta congelada", "Pao de alho congelado", "Pao de queijo recheado congelado", "Sorvete de massa", 
    "Sorvete de palito", "Sorvete","Acai congelado", "Polenta congelada", "Legumes congelados", "Frutas congeladas", "Pure de batata congelado", "Pao frances congelado", "Pao de forma congelado", "Pao integral congelado", 
    "Pao sirio congelado", "Pao de centeio congelado"
]

enlatados = [
    "Milho", "Ervilha", "Seleta de legumes", "Sardinha", "Atum", "Molho de tomate", "Extrato de tomate", "Palmito", "Azeitona", "Seleta de legumes"
]

hortifruti = [
    "Batata", "Cebola", "Tomate", "Alho", "Cenoura", "Abobora", "Chuchu", "Pimentao", "Alface", "Repolho",
    "Couve", "Brocolis", "Banana", "Maca", "Laranja", "Limao", "Mamao", "Abacaxi", "Melancia", "Melao",
    "Uva", "Morango", "Kiwi", "Pera", "Manga", "Goiaba", "Caju", "Maracuja", "Acerola", "Coco",
    "Pessego", "Ameixa", "Figo", "Caqui", "Jabuticaba", "Cupuacu", "Acai", "Guarana", "Pinhao", "Pepino",
    "Rabanete", "Beterraba", "Inhame", "Mandioca", "Batata-doce", "Gengibre", "Coentro", "Salsinha"
]

padaria = [
    "Pao", "Biscoito", "Bolacha", "Pao de queijo", "Pizza", "Lasanha", "Pao integral", "Pao frances", "Pao de forma", "Pao sirio",
    "Pao de centeio", "Pao de aveia", "Pao de milho", "Pao de batata", "Pao de queijo congelado", "Pao de alho", "Biscoito recheado", "Biscoito agua e sal", "Biscoito cream cracker", "Biscoito de polvilho",
    "Biscoito de aveia", "Biscoito de gengibre", "Biscoito de chocolate", "Biscoito de coco", "Biscoito de maisena", "Biscoito de nata", "Pao integral", "Pao frances", "Pao de forma", "Pao sirio", 
    "Pao de centeio", "Pao de aveia", "Pao de milho", "Pao de batata", "Pao de queijo congelado", "Pao de alho"
]

utensilios = [
    "Panela de pressao", "Panela antiaderente", "Panela de ferro", "Panela de barro", "Frigideira", "Chaleira", "Liquidificador", "Batedeira", "Processador de alimentos", "Torradeira",
    "Forno eletrico", "Air fryer", "Cafeteira", "Jarra eletrica", "Faca de cozinha", "Tabua de corte", "Escorredor de louca", "Espatula de silicone", "Colher de pau", "Ralador de queijo",
    "Descascador de legumes", "Assadeira", "Forma de bolo", "Forma de gelo", "Garrafa termica", "Caneca", "Prato", "Tigela", "Talheres", "Guardanapo de pano",
    "Toalha de mesa", "Jogo americano", "Vaso decorativo", "Porta-temperos", "Porta-sabao", "Escova de limpeza", "Rodo de pia", "Vassoura de pia", "Organizador de gavetas", "Cabide",
    "Caderno", "Caneta", "Lapis", "Borracha", "Apontador", "Regua", "Estojo", "Mochila", "Cola branca", "Tesoura"
]

marcas = [
    "Coca-Cola", "Pepsi", "Antarctica", "Sadia", "Perdigao",
    "Seara", "Friboi", "Qualy", "Vigor", "Itambe", "Pia", "Codil",
    "Tres Coracoes", "Pilao", "Melitta", "Nescafe", "Nestle", "Garoto",
    "Lacta", "Bauducco", "Pullman", "Wickbold", "Panco", "Yoki", "Arcor",
    "Unilever", "Omo", "Brillante", "Dove", "Rexona", "Seda", "Clear",
    "Colgate", "Oral-B", "Close-Up", "Palmolive", "Bombril", "Veja",
    "Ajax", "Dettol", "Johnson & Johnson", "Neve", "Kibon", "Ferrero",
    "Nutella", "Tang", "Del Valle", "Maguary", "Jandaia", "Elege",
    "Coca Cola", "Sadia", "Codil", "3Coracoes", "3M", "Ajinomoto", "Bauducco", "Bettanin", "Nivea", "BIC",
    "Bimbo", "Camil", "Colgate", "Elma Chips", "Danone", "Nestle", "Arisco", "Aminas", "Skol", "Perdigao",
    "Havaianas", "Bombril", "Goya", "Soya", "Yoki", "Italac", "Kitano", "Kibon", "Hellmann's", "Tio Joao",
    "Pedigree", "Friboi", "Emulzint", "Itaipava", "Pepsico", "Coca", "Suvini", "Ipanema", "Fanta", "Lacta",
    "Ovomaltine", "Brahma", "Barilla", "Tang", "Sadia", "Tetra Pak", "Fruta Pura", "Del Valle", "Marge",
    "Nescau", "Halls", "JBS", "Nescafe", "Rei do Mate", "Lays", "Kraft", "Marilan", "Seara",
    "Extra", "Motta", "Trident", "Mister Sheik", "Brahma", "Ambev", "Vivo", "Huggies",
    "Dove", "Arcor", "Tetra", "Nestle", "Oster", "Tetra", "Simonsen", "Fermel", "Polenghi",
    "Sadia", "Cargill", "Itambe", "Flormel", "Kraft Heinz", "Santa Helena", "Suavipan", "Santo Antonio",
    "Mustela", "Sabores", "Oro", "Schweppes", "Vovo Alzira", "Carmen", "Salada", "Panco", "Pedigree",
    "Dona Benta", "Paladar", "Queijo Colonial", "Chevrolet", "Vila Real",
    "Orbe", "Jafra", "Motrin", "Renata", "Bongrain", "Piracanjuba", "Jadlog", "Estrela", "Lux", "Fini",
    "Parmalat", "Pavao", "Hershey's", "Aviacao", "Pernod Ricard", "Granja Sao Jose", "Chester",
    "Ameixa", "Ype", "Embalagem", "Mochilao", "Acai Sucos", "Pequeno Principe", "Ponde", "Astra", 
    "Hotdog", "Cremogema", "Superbonder", "Vitao", "Sapore", "Alca", "Citrus", "Delmiro",
    "Rochedo", "Passoca", "Coqueiro", "Green Valley", "Renner", "Nordestino",
    "Fazenda", "Bolos Finos", "Climobag", "Petra", "Biscuit", "D'Agua", "Unilever", "Sodia", "General Mills",
    "Mondelez", "Abdala", "Fazenda Santa", "Ostaria", "Ducoco", "Fiorito", "Ole", "Goia", "Urucu",
    "Leco", "Plaza", "Fit Food", "Bagaggio", "Pao de Acucar", "Arizonia", "Indaia", "Q.A",
    "Marta Rocha", "Shiseido", "Lucarini", "Bis", "Hersheys", "Proteina","Sanremo", "Lide", 
    "Club Social","Sorriso", "Tic Tac", "Chocolandia", "Nugget", "Borges", "Nestle", "Bombril", "Gallo",
    "Gela Pet", "Borges", "Serra", "Cia da Terra", "Peca Gourmet", "Pervical", "Ambev", "Caim", "Ricart",
    "Coca Cola", "Cica", "Gafisa", "Acerola", "Eagle", "Tigrao", "Sun", "Cavallini"

]
# Função para gerar um produto aleatorio e o id da sua categoria

def gerar_produto_categoria():
    produto = random.choice(produtos)
    marca = random.choice(marcas)
    produtomarca = f"{produto} {marca}"
    fake = Faker() #variavel para usar Faker
    data_criacao = fake.date_time_between(start_date='-14y',end_date='now') 
    if produto in mercearia:
        id_categoria = 1
    elif produto in produtos_limpeza:
        id_categoria = 2
    elif produto in  higiene_pessoal:
        id_categoria = 3   
    elif produto in  bebidas_nao_alcoolicas:
        id_categoria = 4
    elif produto in  bebidas_alcoolicas:
        id_categoria = 5
    elif produto in  laticinios:
        id_categoria = 6
    elif produto in  carnes:
        id_categoria = 7
    elif produto in  congelados:
        id_categoria = 8
    elif produto in  enlatados:
        id_categoria = 9    
    elif produto in  hortifruti:
        id_categoria = 10 
    elif produto in  padaria:
        id_categoria = 11 
    else: 
        id_categoria = 12
    
    return produto, produtomarca, id_categoria, data_criacao

#SELECIONAR O QUANTIDADE DE PRODUTOS A SEREM GERADOS
dados_produto = [gerar_produto_categoria() for _ in range(quantidade_produtos_gerados)]

df_produtos = pd.DataFrame(dados_produto, columns=['NOME_ALT', 'NOME', 'ID_CATEGORIA', 'DATA_CRIACAO'])

lista_produtos = [item[0] for item in dados_produto]

# Intervalos de preços (em reais) para cada produto, baseados em valores de mercado no Brasil
intervalos_precos = {
    "Ovos": (8.00,13.00),            # preco por duzia
    "Arroz": (5.00, 10.00),          # Preço por kg
    "Feijao": (8.00, 15.00),         # Preço por kg
    "Acucar": (3.50, 6.00),          # Preço por kg
    "Sal": (2.00, 4.00),             # Preço por kg
    "Oleo de soja": (7.00, 12.00),   # Preço por litro
    "Cafe": (15.00, 30.00),          # Preço por kg
    "Farinha de trigo": (4.00, 8.00), # Preço por kg
    "Macarrao": (3.00, 6.00),        # Preço por 500g
    "Molho de tomate": (2.50, 5.00), # Preço por lata
    "Farinha de mandioca": (5.00, 10.00), # Preço por kg
    "Polenta": (4.00, 8.00),         # Preço por kg
    "Lentilha": (6.00, 12.00),       # Preço por kg
    "Grao-de-bico": (7.00, 14.00),   # Preço por kg
    "Soja": (5.00, 10.00),           # Preço por kg
    "Aveia": (6.00, 12.00),          # Preço por kg
    "Granola": (10.00, 20.00),       # Preço por kg
    "Barra de cereal": (1.50, 3.00), # Preço por unidade
    "Pipoca": (2.00, 5.00),          # Preço por pacote
    "Amendoim": (8.00, 15.00),       # Preço por kg
    "Castanha": (20.00, 40.00),      # Preço por kg
    "Nozes": (30.00, 60.00),         # Preço por kg
    "Uva passa": (10.00, 20.00),     # Preço por kg
    "Azeite": (15.00, 30.00),        # Preço por litro
    "Vinagre": (3.00, 6.00),         # Preço por litro
    "Fermento em po": (2.00, 4.00),  # Preço por pacote
    "Gelatina": (1.50, 3.00),        # Preço por caixa
    "Pudim": (5.00, 10.00),          # Preço por unidade
    "Leite condensado": (4.00, 8.00), # Preço por lata
    "Creme de leite": (3.00, 6.00),  # Preço por caixa
    "Chantilly": (5.00, 10.00),      # Preço por lata
    "Coco ralado": (4.00, 8.00),     # Preço por pacote
    "Canela": (3.00, 6.00),          # Preço por pacote
    "Fermento biologico": (2.00, 4.00), # Preço por unidade
    "Tapioca": (5.00, 10.00),        # Preço por kg
    "Geleia": (4.00, 8.00),          # Preço por pote
    "Mel": (10.00, 20.00),           # Preço por kg
    "Pasta de amendoim": (8.00, 15.00), # Preço por pote
    "Sopa instantanea": (2.00, 5.00), # Preço por unidade
    "Filtro de cafe": (5.00, 10.00), # Preço por pacote
    "Cereal matinal": (8.00, 15.00), # Preço por caixa
    "Achocolatado": (6.00, 12.00),   # Preço por kg
    "Extrato de tomate": (2.50, 5.00), # Preço por lata
    "Maionese": (4.00, 8.00),        # Preço por pote
    "Mostarda": (3.00, 6.00),        # Preço por pote
    "Ketchup": (3.00, 6.00),         # Preço por pote
    "Tempero pronto": (2.00, 4.00),  # Preço por pacote
    "Caldo de carne": (1.50, 3.00),  # Preço por tablete
    "Caldo de galinha": (1.50, 3.00), # Preço por tablete
    "Seleta de legumes": (3.00, 6.00), # Preço por lata
    "Farinha de rosca": (3.00, 6.00), # Preço por pacote
    "Fermento quimico": (2.00, 4.00), # Preço por pacote
    "Cacau em po": (6.00, 12.00),    # Preço por kg
    "Cha mate": (5.00, 10.00),       # Preço por pacote
    "Cha verde": (5.00, 10.00),      # Preço por pacote
    "Cha de camomila": (5.00, 10.00), # Preço por pacote
    "Cha de hortela": (5.00, 10.00), # Preço por pacote
    "Cafe soluvel": (10.00, 20.00),  # Preço por pote
    "Cafe em capsula": (15.00, 30.00), # Preço por caixa
    "Cafe descafeinado": (10.00, 20.00), # Preço por pote
    "Oleo de girassol": (7.00, 12.00), # Preço por litro
    "Oleo de milho": (7.00, 12.00),  # Preço por litro
    "Oleo de canola": (7.00, 12.00), # Preço por litro
    "Oleo de coco": (10.00, 20.00),  # Preço por litro
    "Oleo de linhaca": (15.00, 30.00), # Preço por litro
    "Oleo de gergelim": (15.00, 30.00), # Preço por litro
    "Oleo de amendoim": (10.00, 20.00), # Preço por litro
    "Oleo de algodao": (7.00, 12.00), # Preço por litro
    "Oleo de palma": (7.00, 12.00),  # Preço por litro
    "Oleo de abacate": (15.00, 30.00), # Preço por litro
    "Arroz integral": (6.00, 12.00), # Preço por kg
    "Arroz arboreo": (8.00, 15.00),  # Preço por kg
    "Arroz negro": (10.00, 20.00),   # Preço por kg
    "Arroz vermelho": (10.00, 20.00), # Preço por kg
    "Arroz cateto": (8.00, 15.00),   # Preço por kg
    "Arroz jasmine": (10.00, 20.00), # Preço por kg
    "Arroz basmati": (12.00, 25.00), # Preço por kg
    "Arroz parboilizado": (6.00, 12.00), # Preço por kg
    "Arroz selvagem": (15.00, 30.00), # Preço por kg
    "Arroz 7 graos": (10.00, 20.00), # Preço por kg
    "Farinha de milho": (4.00, 8.00), # Preço por kg
    "Farinha de aveia": (6.00, 12.00), # Preço por kg
    "Farinha de centeio": (6.00, 12.00), # Preço por kg
    "Farinha de cevada": (6.00, 12.00), # Preço por kg
    "Farinha de coco": (8.00, 15.00), # Preço por kg
    "Farinha de amendoas": (15.00, 30.00), # Preço por kg
    "Farinha de arroz": (6.00, 12.00), # Preço por kg
    "Farinha de grao-de-bico": (7.00, 14.00), # Preço por kg
    "Farinha de quinoa": (10.00, 20.00), # Preço por kg
    "Farinha de linhaca": (10.00, 20.00), # Preço por kg
    "Farinha de tapioca": (5.00, 10.00), # Preço por kg
    "Farinha de banana verde": (8.00, 15.00), # Preço por kg
    "Farinha de maracuja": (10.00, 20.00), # Preço por kg
    "Farinha de berinjela": (8.00, 15.00), # Preço por kg
    "Farinha de soja": (6.00, 12.00), # Preço por kg
    "Farinha de trigo integral": (5.00, 10.00), # Preço por kg
    "Farinha de trigo sarraceno": (8.00, 15.00), # Preço por kg
    "Farinha de trigo sem gluten": (10.00, 20.00), # Preço por kg
    "Farinha de trigo especial": (6.00, 12.00), # Preço por kg
    "Farinha de trigo tipo 1": (4.00, 8.00), # Preço por kg
    "Salgadinho": (2.00, 5.00),      # Preço por pacote
    "Chocolate": (5.00, 10.00),      # Preço por barra
    "Balas": (1.00, 3.00),           # Preço por pacote
    "Goma de mascar": (2.00, 5.00),  # Preço por pacote
    "Pipoca doce": (3.00, 6.00),     # Preço por pacote
    "Pipoca salgada": (3.00, 6.00),  # Preço por pacote
    "Pipoca de micro-ondas": (4.00, 8.00), # Preço por pacote
    "Pipoca caramelizada": (5.00, 10.00), # Preço por pacote
    "Pipoca com manteiga": (5.00, 10.00), # Preço por pacote
    "Pipoca com queijo": (5.00, 10.00), # Preço por pacote
    "Pipoca com bacon": (5.00, 10.00), # Preço por pacote
    "Pipoca colorida": (5.00, 10.00), # Preço por pacote
    "Pipoca zero acucar": (5.00, 10.00), # Preço por pacote
    "Pipoca zero gordura": (5.00, 10.00), # Preço por pacote
    "Sopa instantanea": (2.00, 5.00), # Preço por unidade
    "Farinha de trigo tipo 1": (4.00, 8.00), # Preço por kg
    "Filtro de cafe": (5.00, 10.00), # Preço por pacote
    "Cereal matinal": (8.00, 15.00), # Preço por caixa
    "Achocolatado": (6.00, 12.00),   # Preço por kg
    "Leite em po": (10.00, 20.00),   # Preço por lata
    "Leite de soja": (5.00, 10.00),  # Preço por litro
    "Leite de amendoas": (8.00, 15.00), # Preço por litro
    "Leite de coco": (6.00, 12.00),  # Preço por litro
    "Leite de aveia": (6.00, 12.00), # Preço por litro
    "Leite sem lactose": (5.00, 10.00), # Preço por litro
    "Biscoito recheado": (2.00, 5.00), # Preço por pacote
    "Biscoito agua e sal": (2.00, 5.00), # Preço por pacote
    "Biscoito cream cracker": (2.00, 5.00), # Preço por pacote
    "Biscoito de polvilho": (3.00, 6.00), # Preço por pacote
    "Biscoito de aveia": (3.00, 6.00), # Preço por pacote
    "Biscoito de gengibre": (3.00, 6.00), # Preço por pacote
    "Biscoito de chocolate": (3.00, 6.00), # Preço por pacote
    "Biscoito de coco": (3.00, 6.00), # Preço por pacote
    "Biscoito de maisena": (3.00, 6.00), # Preço por pacote
    "Biscoito de nata": (3.00, 6.00),  # Preço por pacote
    "Detergente": (2.00, 5.00),                  # Preço por litro
    "Amaciante": (8.00, 15.00),                  # Preço por litro
    "Agua sanitaria": (3.00, 6.00),              # Preço por litro
    "Desinfetante": (5.00, 10.00),               # Preço por litro
    "Esponja de lavar louca": (1.50, 3.00),      # Preço por unidade
    "Sacola plastica": (0.50, 1.50),             # Preço por pacote
    "Guardanapo": (2.00, 4.00),                  # Preço por pacote
    "Toalha de papel": (3.00, 6.00),             # Preço por rolo
    "Sabao em po": (8.00, 15.00),                # Preço por kg
    "Limpador multiuso": (5.00, 10.00),          # Preço por litro
    "Detergente neutro": (2.00, 5.00),           # Preço por litro
    "Detergente concentrado": (3.00, 6.00),      # Preço por litro
    "Amaciante perfumado": (8.00, 15.00),        # Preço por litro
    "Amaciante hipoalergenico": (10.00, 20.00),  # Preço por litro
    "Desinfetante para banheiro": (5.00, 10.00), # Preço por litro
    "Desinfetante para cozinha": (5.00, 10.00),  # Preço por litro
    "Limpador para vidros": (5.00, 10.00),       # Preço por litro
    "Limpador para pisos": (5.00, 10.00),        # Preço por litro
    "Lustra-moveis": (5.00, 10.00),              # Preço por litro
    "Saco de lixo": (5.00, 10.00),               # Preço por pacote
    "Saco de lixo reciclavel": (6.00, 12.00),    # Preço por pacote
    "Saco de lixo biodegradavel": (7.00, 14.00), # Preço por pacote
    "Vassoura": (10.00, 20.00),                  # Preço por unidade
    "Rodo": (5.00, 10.00),                       # Preço por unidade
    "Pa de lixo": (5.00, 10.00),                 # Preço por unidade
    "Balde": (10.00, 20.00),                     # Preço por unidade
    "Esponja de aco": (2.00, 4.00),              # Preço por unidade
    "Esponja multiuso": (3.00, 6.00),            # Preço por unidade
    "Luvas de limpeza": (5.00, 10.00),           # Preço por par
    "Alcool em gel": (10.00, 20.00),             # Preço por litro
    "Alcool liquido": (8.00, 15.00),             # Preço por litro
    "Limpa-vidros": (5.00, 10.00),               # Preço por litro
    "Limpa-forno": (5.00, 10.00),                # Preço por litro
    "Limpa-carpete": (5.00, 10.00),              # Preço por litro
    "Limpa-azulejo": (5.00, 10.00),              # Preço por litro
    "Limpa-inox": (5.00, 10.00),                 # Preço por litro
    "Limpa-pisos": (5.00, 10.00),                # Preço por litro
    "Limpa-tapetes": (5.00, 10.00),              # Preço por litro
    "Limpa-estofados": (5.00, 10.00),            # Preço por litro
    "Desengordurante": (5.00, 10.00),            # Preço por litro
    "Removedor de manchas": (5.00, 10.00),       # Preço por litro
    "Cera para pisos": (10.00, 20.00),           # Preço por litro
    "Cera para moveis": (10.00, 20.00),          # Preço por litro
    "Cera para carro": (10.00, 20.00),           # Preço por litro
    "Inseticida": (5.00, 10.00),                 # Preço por litro
    "Repelente de insetos": (5.00, 10.00),       # Preço por litro
    "Aromatizador de ambientes": (5.00, 10.00),  # Preço por unidade
    "Difusor de aromas": (10.00, 20.00),         # Preço por unidade
    "Velas aromaticas": (5.00, 10.00),            # Preço por unidade
    "Sabonete": (2.50, 10.00),
    "Shampoo": (10.00, 40.00),
    "Condicionador": (10.00, 40.00),
    "Papel higienico": (5.00, 25.00),
    "Creme dental": (3.50, 15.00),
    "Escova de dentes": (5.00, 20.00),
    "Desodorante": (7.00, 25.00),
    "Absorvente": (5.00, 20.00),
    "Lamina de barbear": (5.00, 30.00),
    "Creme hidratante": (10.00, 50.00),
    "Desodorante roll-on": (8.00, 18.00),
    "Desodorante aerosol": (10.00, 30.00),
    "Sabonete liquido": (8.00, 25.00),
    "Sabonete em barra": (2.50, 12.00),
    "Creme para barbear": (8.00, 25.00),
    "Locao pos-barba": (12.00, 50.00),
    "Protetor solar": (30.00, 80.00),
    "Hidratante corporal": (15.00, 60.00),
    "Lenco umedecido": (6.00, 25.00),
    "Absorvente noturno": (6.00, 22.00),
    "Absorvente diario": (5.00, 18.00),
    "Cotonete": (3.00, 10.00),
    "Fio dental": (5.00, 20.00),
    "Enxaguante bucal": (10.00, 35.00),
    "Pasta de dente clareadora": (10.00, 25.00),
    "Escova de dentes infantil": (5.00, 15.00),
    "Escova de dentes eletrica": (50.00, 200.00),
    "Creme para pentear": (10.00, 35.00),
    "Gel para cabelo": (8.00, 30.00),
    "Mascara capilar": (15.00, 60.00),
    "Condicionador reparador": (12.00, 50.00),
    "Shampoo anticaspa": (15.00, 45.00),
    "Shampoo infantil": (10.00, 35.00),
    "Sabonete intimo": (8.00, 30.00),
    "Creme para assaduras": (10.00, 40.00),
    "Oleo para bebe": (15.00, 50.00),
    "Lenco de papel": (3.00, 15.00),
    "Papel higienico perfumado": (8.00, 30.00),
    "Papel higienico biodegradavel": (10.00, 35.00),
    "Sabonete antibacteriano": (5.00, 20.00),
    "Desinfetante para as maos": (10.00, 30.00),
    "Creme para os pes": (12.00, 40.00),
    "Esmalte de unha": (5.00, 25.00),
    "Removedor de esmalte": (5.00, 20.00),
    "Cera depilatoria": (10.00, 40.00),
    "Lamina de barbear descartavel": (5.00, 25.00),
    "Creme para rugas": (30.00, 100.00),
    "Protetor labial": (10.00, 30.00),
    "Po compacto": (20.00, 80.00),
    "Batom": (15.00, 50.00),
    "Base liquida": (20.00, 90.00),
    "Demaquilante": (15.00, 50.00),
    "Creme para olheiras": (20.00, 80.00),
    "Creme para acne": (15.00, 60.00),
    "Protetor solar facial": (30.00, 100.00),
    "Shampoo seco": (15.00, 45.00),
    "Condicionador sem enxague": (12.00, 40.00),
    "Gel para sobrancelha": (15.00, 50.00),
    "Creme para maos": (10.00, 35.00),
    "Sabonete esfoliante": (8.00, 30.00),
    "Refrigerante": (5.00, 10.00),          # Preço por 2L
    "Suco": (8.00, 15.00),                  # Preço por 1L
    "Agua mineral": (2.00, 5.00),           # Preço por 1,5L
    "Cha": (3.00, 6.00),                    # Preço por caixa com 10 saquinhos
    "Achocolatado": (6.00, 12.00),          # Preço por 400g
    "Cafe": (10.00, 20.00),                 # Preço por 500g
    "Leite": (4.00, 8.00),                  # Preço por 1L
    "Energetico": (6.00, 12.00),            # Preço por lata
    "Cerveja sem alcool": (3.00, 6.00),     # Preço por lata
    "Cerveja zero alcool": (3.00, 6.00),     # Preço por lata
    "Cerveja": (3.00, 6.00),              # Preço por lata ou garrafa
    "Vinho": (20.00, 50.00),              # Preço por garrafa
    "Vodka": (30.00, 80.00),              # Preço por garrafa
    "Whisky": (50.00, 150.00),            # Preço por garrafa
    "Rum": (30.00, 70.00),                # Preço por garrafa
    "Gin": (40.00, 100.00),               # Preço por garrafa
    "Tequila": (50.00, 120.00),           # Preço por garrafa
    "Cachaca": (15.00, 40.00),            # Preço por garrafa
    "Licor": (20.00, 60.00),              # Preço por garrafa
    "Champagne": (50.00, 150.00),         # Preço por garrafa
    "Espumante": (30.00, 80.00),          # Preço por garrafa
    "Sidra": (20.00, 50.00),              # Preço por garrafa
    "Conhaque": (40.00, 100.00),          # Preço por garrafa
    "Vermute": (20.00, 50.00),            # Preço por garrafa
    "Sake": (30.00, 70.00),               # Preço por garrafa
    "Absinto": (50.00, 120.00),           # Preço por garrafa
    "Catuaba": (15.00, 30.00),            # Preço por garrafa
    "Pinga artesanal": (20.00, 50.00),    # Preço por garrafa
    "Vinho tinto": (20.00, 60.00),        # Preço por garrafa
    "Vinho branco": (20.00, 60.00),       # Preço por garrafa
    "Vinho rose": (20.00, 60.00),         # Preço por garrafa
    "Vinho seco": (20.00, 60.00),         # Preço por garrafa
    "Vinho suave": (20.00, 60.00),        # Preço por garrafa
    "Cerveja artesanal": (10.00, 25.00),  # Preço por garrafa
    "Cerveja de trigo": (8.00, 15.00),    # Preço por garrafa
    "Cerveja IPA": (10.00, 20.00),        # Preço por garrafa
    "Cerveja stout": (10.00, 20.00),      # Preço por garrafa
    "Cerveja pilsen": (5.00, 10.00),      # Preço por garrafa
    "Cerveja light": (5.00, 10.00),       # Preço por garrafa
    "Cerveja de milho": (5.00, 10.00),     # Preço por garrafa
    "Leite": (4.00, 8.00),
    "Queijo": (20.00, 80.00),
    "Manteiga": (10.00, 30.00),
    "Margarina": (5.00, 15.00),
    "Iogurte": (3.00, 12.00),
    "Requeijao": (6.00, 15.00),
    "Creme de leite": (3.00, 10.00),
    "Leite condensado": (5.00, 12.00),
    "Chantilly": (8.00, 20.00),
    "Fermento biologico": (2.00, 8.00),
    "Leite em po": (10.00, 40.00),
    "Leite de soja": (8.00, 18.00),
    "Leite de amendoas": (12.00, 25.00),
    "Leite de coco": (5.00, 15.00),
    "Leite de aveia": (10.00, 22.00),
    "Leite sem lactose": (6.00, 12.00),
    "Iogurte grego": (4.00, 12.00),
    "Iogurte natural": (3.00, 10.00),
    "Iogurte com frutas": (4.00, 12.00),
    "Iogurte zero lactose": (5.00, 14.00),
    "Queijo minas": (30.00, 70.00),
    "Queijo prato": (25.00, 60.00),
    "Queijo mussarela": (20.00, 55.00),
    "Queijo parmesao": (50.00, 120.00),
    "Queijo provolone": (40.00, 90.00),
    "Queijo cottage": (10.00, 30.00),
    "Queijo ricota": (8.00, 25.00),
    "Queijo gorgonzola": (50.00, 100.00),
    "Queijo brie": (70.00, 150.00),
    "Queijo coalho": (15.00, 40.00),
    "Yakult": (8.00, 20.00),
    "Danoninho": (6.00, 15.00),
    "Carne bovina": (30.00, 60.00),          # Preço por kg
    "Frango": (15.00, 30.00),                # Preço por kg
    "Peixe": (20.00, 50.00),                 # Preço por kg
    "Salsicha": (10.00, 20.00),              # Preço por kg
    "Presunto": (20.00, 40.00),              # Preço por kg
    "Linguica": (15.00, 30.00),              # Preço por kg
    "Carne moida": (25.00, 50.00),           # Preço por kg
    "Costela": (30.00, 60.00),               # Preço por kg
    "Bacon": (20.00, 40.00),                 # Preço por kg
    "Hamburguer": (15.00, 30.00),            # Preço por kg
    "Presunto cozido": (20.00, 40.00),       # Preço por kg
    "Presunto defumado": (25.00, 50.00),     # Preço por kg
    "Peito de peru": (20.00, 40.00),         # Preço por kg
    "Salame": (25.00, 50.00),                # Preço por kg
    "Mortadela": (15.00, 30.00),             # Preço por kg
    "Rosbife": (30.00, 60.00),               # Preço por kg
    "Carne seca": (40.00, 80.00),            # Preço por kg
    "Charque": (35.00, 70.00),               # Preço por kg
    "Bacalhau": (50.00, 100.00),             # Preço por kg
    "Tilapia": (20.00, 40.00),               # Preço por kg
    "Salmao": (60.00, 120.00),               # Preço por kg
    "Camarao": (50.00, 100.00),              # Preço por kg
    "Lula": (40.00, 80.00),                  # Preço por kg
    "Polvo": (50.00, 100.00),                # Preço por kg
    "Ostra": (30.00, 60.00),                 # Preço por kg
    "Mexilhao": (20.00, 40.00),              # Preço por kg
    "Sushi": (10.00, 20.00),                 # Preço por porção
    "Temaki": (15.00, 30.00),                # Preço por unidade
    "Salsicha hot dog": (10.00, 20.00),      # Preço por kg
    "Salsicha para churrasco": (15.00, 30.00), # Preço por kg
    "Salsicha de frango": (12.00, 25.00),    # Preço por kg
    "Salsicha de peru": (12.00, 25.00),      # Preço por kg
    "Salsicha vegetariana": (15.00, 30.00),  # Preço por kg
    "Hamburguer de frango": (15.00, 30.00),  # Preço por kg
    "Hamburguer vegetariano": (20.00, 40.00), # Preço por kg
    "Hamburguer de soja": (18.00, 35.00),    # Preço por kg
    "Hamburguer de peixe": (20.00, 40.00),   # Preço por kg
    "Hamburguer artesanal": (25.00, 50.00),   # Preço por kg
    "Pizza congelada": (15.00, 30.00),              # Preço por unidade
    "Lasanha congelada": (20.00, 40.00),            # Preço por unidade
    "Pao de queijo congelado": (10.00, 20.00),      # Preço por pacote
    "Hamburguer congelado": (15.00, 30.00),         # Preço por kg
    "Batata frita congelada": (10.00, 20.00),       # Preço por kg
    "Nuggets de frango": (15.00, 30.00),            # Preço por kg
    "Empanado de frango": (20.00, 40.00),           # Preço por kg
    "Empanado de peixe": (25.00, 50.00),            # Preço por kg
    "Escondidinho congelado": (15.00, 30.00),       # Preço por unidade
    "Feijoada congelada": (20.00, 40.00),           # Preço por unidade
    "Risoto congelado": (15.00, 30.00),             # Preço por unidade
    "Yakisoba congelado": (10.00, 20.00),           # Preço por unidade
    "Coxinha congelada": (5.00, 10.00),             # Preço por unidade
    "Kibe congelado": (5.00, 10.00),                # Preço por unidade
    "Pastel congelado": (5.00, 10.00),              # Preço por unidade
    "Torta congelada": (15.00, 30.00),              # Preço por unidade
    "Pao de alho congelado": (10.00, 20.00),        # Preço por pacote
    "Pao de queijo recheado congelado": (12.00, 25.00), # Preço por pacote
    "Sorvete de massa": (10.00, 20.00),             # Preço por pote
    "Sorvete de palito": (2.00, 5.00),              # Preço por unidade
    "Sorvete": (10.00, 20.00),                      # Preço por pote
    "Acai congelado": (15.00, 30.00),               # Preço por kg
    "Polenta congelada": (10.00, 20.00),            # Preço por kg
    "Legumes congelados": (8.00, 15.00),            # Preço por kg
    "Frutas congeladas": (10.00, 20.00),            # Preço por kg
    "Pure de batata congelado": (10.00, 20.00),     # Preço por kg
    "Pao frances congelado": (5.00, 10.00),         # Preço por unidade
    "Pao de forma congelado": (5.00, 10.00),        # Preço por pacote
    "Pao integral congelado": (6.00, 12.00),        # Preço por pacote
    "Pao sirio congelado": (8.00, 15.00),           # Preço por pacote
    "Pao de centeio congelado": (8.00, 15.00),       # Preço por pacote
    "Milho": (3.00, 6.00),              # Preço por lata
    "Ervilha": (3.00, 6.00),            # Preço por lata
    "Seleta de legumes": (4.00, 8.00),  # Preço por lata
    "Sardinha": (5.00, 10.00),          # Preço por lata
    "Atum": (8.00, 15.00),              # Preço por lata
    "Molho de tomate": (2.50, 5.00),    # Preço por lata
    "Extrato de tomate": (3.00, 6.00),  # Preço por lata
    "Palmito": (10.00, 20.00),          # Preço por lata
    "Azeitona": (5.00, 10.00),          # Preço por lata
    "Seleta de legumes": (4.00, 8.00),   # Preço por lata
    "Batata": (3.00, 6.00),
    "Cebola": (2.50, 5.50),
    "Tomate": (4.00, 10.00),
    "Alho": (15.00, 40.00),
    "Cenoura": (2.00, 6.00),
    "Abobora": (3.00, 8.00),
    "Chuchu": (2.00, 5.00),
    "Pimentao": (5.00, 12.00),
    "Alface": (2.00, 5.00),
    "Repolho": (3.00, 7.00),
    "Couve": (2.50, 6.00),
    "Brocolis": (5.00, 15.00),
    "Banana": (3.00, 7.00),
    "Maca": (5.00, 12.00),
    "Laranja": (2.50, 7.00),
    "Limao": (2.00, 6.00),
    "Mamao": (4.00, 10.00),
    "Abacaxi": (5.00, 12.00),
    "Melancia": (6.00, 15.00),
    "Melao": (6.00, 14.00),
    "Uva": (7.00, 20.00),
    "Morango": (8.00, 20.00),
    "Kiwi": (10.00, 25.00),
    "Pera": (6.00, 14.00),
    "Manga": (4.00, 12.00),
    "Goiaba": (3.00, 8.00),
    "Caju": (5.00, 12.00),
    "Maracuja": (6.00, 15.00),
    "Acerola": (7.00, 18.00),
    "Coco": (4.00, 10.00),
    "Pessego": (6.00, 14.00),
    "Ameixa": (8.00, 20.00),
    "Figo": (10.00, 25.00),
    "Caqui": (5.00, 12.00),
    "Jabuticaba": (8.00, 18.00),
    "Cupuacu": (6.00, 14.00),
    "Acai": (10.00, 30.00),
    "Guarana": (6.00, 15.00),
    "Pinhao": (10.00, 25.00),
    "Pepino": (3.00, 6.00),
    "Rabanete": (4.00, 8.00),
    "Beterraba": (3.00, 7.00),
    "Inhame": (5.00, 12.00),
    "Mandioca": (4.00, 10.00),
    "Batata-doce": (3.50, 7.50),
    "Gengibre": (8.00, 18.00),
    "Coentro": (1.50, 4.00),
    "Salsinha": (1.50, 4.00),
    "Pao": (5.00, 10.00),                  # Preço por kg
    "Biscoito": (3.00, 6.00),              # Preço por pacote
    "Bolacha": (3.00, 6.00),               # Preço por pacote
    "Pao de queijo": (10.00, 20.00),       # Preço por kg
    "Pizza": (15.00, 30.00),               # Preço por unidade
    "Lasanha": (20.00, 40.00),             # Preço por unidade
    "Pao integral": (6.00, 12.00),         # Preço por kg
    "Pao frances": (5.00, 10.00),          # Preço por unidade
    "Pao de forma": (5.00, 10.00),         # Preço por pacote
    "Pao sirio": (8.00, 15.00),            # Preço por pacote
    "Pao de centeio": (6.00, 12.00),       # Preço por kg
    "Pao de aveia": (6.00, 12.00),         # Preço por kg
    "Pao de milho": (5.00, 10.00),         # Preço por kg
    "Pao de batata": (5.00, 10.00),        # Preço por kg
    "Pao de queijo congelado": (12.00, 25.00), # Preço por kg
    "Pao de alho": (10.00, 20.00),         # Preço por unidade
    "Biscoito recheado": (4.00, 8.00),     # Preço por pacote
    "Biscoito agua e sal": (3.00, 6.00),   # Preço por pacote
    "Biscoito cream cracker": (3.00, 6.00), # Preço por pacote
    "Biscoito de polvilho": (4.00, 8.00),  # Preço por pacote
    "Biscoito de aveia": (4.00, 8.00),     # Preço por pacote
    "Biscoito de gengibre": (4.00, 8.00),  # Preço por pacote
    "Biscoito de chocolate": (4.00, 8.00), # Preço por pacote
    "Biscoito de coco": (4.00, 8.00),      # Preço por pacote
    "Biscoito de maisena": (4.00, 8.00),   # Preço por pacote
    "Biscoito de nata": (4.00, 8.00),       # Preço por pacote
    "Panela de pressao": (100, 300),
    "Panela antiaderente": (40, 150),
    "Panela de ferro": (60, 250),
    "Panela de barro": (40, 120),
    "Frigideira": (30, 120),
    "Chaleira": (20, 70),
    "Liquidificador": (80, 500),
    "Batedeira": (100, 600),
    "Processador de alimentos": (150, 500),
    "Torradeira": (50, 150),
    "Forno eletrico": (150, 700),
    "Air fryer": (200, 1000),
    "Cafeteira": (30, 250),
    "Jarra eletrica": (50, 200),
    "Faca de cozinha": (20, 200),
    "Tabua de corte": (10, 60),
    "Escorredor de louca": (10, 40),
    "Espatula de silicone": (10, 30),
    "Colher de pau": (5, 20),
    "Ralador de queijo": (10, 50),
    "Descascador de legumes": (10, 30),
    "Assadeira": (20, 80),
    "Forma de bolo": (15, 70),
    "Forma de gelo": (5, 20),
    "Garrafa termica": (20, 150),
    "Caneca": (10, 40),
    "Prato": (5, 30),
    "Tigela": (10, 50),
    "Talheres": (10, 50),
    "Guardanapo de pano": (10, 30),
    "Toalha de mesa": (20, 100),
    "Jogo americano": (10, 50),
    "Vaso decorativo": (20, 150),
    "Porta-temperos": (10, 50),
    "Porta-sabao": (10, 30),
    "Escova de limpeza": (5, 30),
    "Rodo de pia": (10, 40),
    "Vassoura de pia": (10, 50),
    "Organizador de gavetas": (15, 100),
    "Cabide": (5, 20),
    "Caderno": (2, 10),
    "Caneta": (1, 5),
    "Lapis": (1, 5),
    "Borracha": (1, 5),
    "Apontador": (1, 10),
    "Regua": (2, 10),
    "Estojo": (5, 30),
    "Mochila": (50, 300),
    "Cola branca": (3, 10),
    "Tesoura": (5, 30)
}

# Função para gerar valores aleatórios dentro dos intervalos
def gerar_valores(produto, intervalos):
    valores = []
    valores_cust = []
    for item in produto:
        min_valor, max_valor = intervalos[item]
        valor = round(random.uniform(min_valor, max_valor), 2)  # Gera valor aleatório e arredonda para 2 casas decimais
        valor_custo = round(((valor/100)*(100-(random.uniform(15, 35)))),2)
        valores.append(valor)
        valores_cust.append(valor_custo)
        
    return valores, valores_cust

# Gerar lista de valores
valores_unitarios, valores_custo = gerar_valores(lista_produtos, intervalos_precos)
df_produtos["PRECO_CUSTO"] = valores_custo
df_produtos["PRECO_UNIT"] = valores_unitarios

#Limpeza de espacos desnecessarios nos dados de entrada
df_produtos = df_produtos.drop_duplicates(subset=['NOME'])
df_produtos = df_produtos.apply(lambda col: col.str.replace(r'\s+', ' ', regex=True).str.strip() if col.dtype == 'object' and col.apply(lambda x: isinstance(x, str)).any() else col)

df_produto = pd.DataFrame({ 

                    "NOME":           df_produtos['NOME'],
                    "ID_CATEGORIA":   df_produtos['ID_CATEGORIA'],
                    "PRECO_CUSTO":    df_produtos["PRECO_CUSTO"],
                    "PRECO_UNIT":     df_produtos["PRECO_UNIT"],
                    "ID_UNID_MED":    1,
                    "DATA_CRIACAO":   df_produtos["DATA_CRIACAO"].dt.date,
                    "DATA_ATLZ":      None,
                    "VERSAO":         1,

})

tipo_dados_produto = {
                'NOME':          VARCHAR(100),
                'ID_CATEGORIA':  INTEGER,
                'PRECO_CUSTO':   NUMERIC(18,2),
                'PRECO_UNIT':    NUMERIC(18,2),
                'ID_UNID_MED':   INTEGER,
                "DATA_CRIACAO":  DATE,
                "DATA_ATLZ":     DATE,
                "VERSAO":        INTEGER,
}

#Função para remover espaços desnecessários em todos os tipos de dados
def strip_all_types(col):
    # Verifica se a coluna é do tipo datetime
    if np.issubdtype(col.dtype, np.datetime64):
        # Se for datetime, retorna a coluna original sem modificações
        return col
    
    col_stripped = col.astype(str).str.replace(r'\s+', ' ', regex=True).str.strip() #Converte a coluna para string, remove espaços extras e aplica strip
    
    #Converte de volta para o tipo original da coluna
    if np.issubdtype(col.dtype, np.integer):  # Se for inteiro
        return pd.to_numeric(col_stripped, errors='coerce').astype(col.dtype)
    elif np.issubdtype(col.dtype, np.floating):  # Se for float
        return pd.to_numeric(col_stripped, errors='coerce').astype(col.dtype)
    elif col.name in ["DATA_CRIACAO", "DATA_ATLZ"]:  # Se for uma coluna de data, Converte valores inválidos (como 'None') para NaT (Not a Time)
        return pd.to_datetime(col_stripped, format='%Y-%m-%d', errors='coerce').dt.date
    else:  # Se for string ou outro tipo
        return col_stripped


#with engine.connect() as conn:
#        conn.execute(text(f"TRUNCATE TABLE PRODUTO"))  #limpa os dados existentes, pode ser removido do codigo
#        conn.commit()
#
#df_produto.to_sql(name='PRODUTO',con=engine,if_exists='append',index=False,dtype=tipo_dados_produto)

with engine.connect() as conn:
    # Consulta os valores existentes na coluna "NOME" da tabela "PRODUTO"
    query = text(f"SELECT {coluna} FROM {tabela}")
    lista_valores_existentes = pd.read_sql(query, conn)[coluna].tolist()
    
    # Filtra os valores que não existem na tabela
    df_produto_filtrado = df_produto[~df_produto["NOME"].isin(lista_valores_existentes)]
    
    # Se houver valores para adicionar
    if not df_produto_filtrado.empty:
        # Limpa os espaços desnecessários
        df_produto_filtrado = df_produto_filtrado.apply(strip_all_types)

        # Define os tipos de dados para a tabela PRODUTO
        tipo_dados_produto = {
            "NOME": VARCHAR(100),
            "ID_CATEGORIA": INTEGER,
            "PRECO_CUSTO": VARCHAR(20),
            "PRECO_UNIT": VARCHAR(20),
            "ID_UNID_MED": INTEGER,
            "DATA_CRIACAO": DATE,
            "DATA_ATLZ": DATE,
            "VERSAO": INTEGER,
        }

        # Exibe os dados que serão adicionados
        print(F"Produtos a serem adicionados: {df_produto_filtrado["NOME"].count()} a partir do ID: {len(lista_valores_existentes)+1}")
        display(df_produto_filtrado)

        # Adiciona os dados ao banco de dados
        df_produto_filtrado.to_sql(
            name=tabela,
            con=engine,
            if_exists="append",
            index=False,
            dtype=tipo_dados_produto
        )

        print("Produtos inseridos com sucesso.")
    else:
        print("Nenhum valor adicionado.")

# Fecha a conexão com o banco de dados
engine.dispose()

#listaprodutosPROVISORIO = list(df_produto["NOME"])
#
#print(listaprodutosPROVISORIO)
#print(len(listaprodutosPROVISORIO))


Conexão com o banco de dados estabelecida com sucesso
Query executada com sucesso
A conexão foi bem-sucedida
Produtos a serem adicionados: 35887 a partir do ID: 62173


,NOME,ID_CATEGORIA,PRECO_CUSTO,PRECO_UNIT,ID_UNID_MED,DATA_CRIACAO,DATA_ATLZ,VERSAO
1,Vinho rose Tetra,5,21.13,29.22,1,2021-02-08,NaT,1
2,Feijoada congelada Pequeno Principe,8,23.83,35.30,1,2023-06-10,NaT,1
4,Arroz Nutella,1,7.10,8.69,1,2024-05-03,NaT,1
6,Fermento em po Borges,1,1.64,2.28,1,2015-08-19,NaT,1
7,Hidratante corporal Kibon,3,14.38,20.53,1,2016-03-15,NaT,1
...,...,...,...,...,...,...,...,...
249852,Gin Bolos Finos,5,58.05,88.19,1,2018-09-01,NaT,1
249863,Iogurte Halls,6,8.47,11.75,1,2018-06-03,NaT,1
249918,Sushi Fini,7,9.96,14.58,1,2014-03-08,NaT,1
249958,Biscoito cream cracker Marge,1,2.59,3.65,1,2014-12-12,NaT,1


Produtos inseridos com sucesso.


In [ ]:
#Script para gerar Tabela Produto

import pandas as pd
import random

produtos = [
    "Arroz", "Feijao", "Acucar", "Sal", "Oleo de soja", "Cafe", "Farinha de trigo", "Leite", "Ovos", "Pao", "Macarrao", "Molho de tomate", "Carne bovina", "Frango", "Peixe", "Salsicha", "Presunto", "Queijo", "Manteiga", "Margarina",
    "Iogurte", "Requeijao", "Biscoito", "Bolacha", "Sabonete", "Shampoo", "Condicionador", "Papel higienico", "Detergente", "Amaciante", "Agua sanitaria", "Desinfetante", "Esponja de lavar louca", "Sacola plastica", "Guardanapo", 
    "Toalha de papel", "Filtro de cafe", "Cereal matinal", "Achocolatado", "Refrigerante", "Suco", "Cerveja", "Vinho", "Agua mineral", "Cha", "Salgadinho", "Chocolate", "Balas", "Goma de mascar", "Sorvete","Milho", "Ervilha", 
    "Seleta de legumes", "Extrato de tomate", "Maionese", "Mostarda", "Ketchup", "Tempero pronto", "Caldo de carne", "Caldo de galinha", "Farinha de mandioca", "Polenta", "Lentilha", "Grao-de-bico", "Soja", "Aveia", "Granola", 
    "Barra de cereal", "Pipoca", "Amendoim", "Castanha", "Nozes", "Uva passa", "Azeite", "Vinagre", "Fermento em po", "Gelatina", "Pudim", "Leite condensado", "Creme de leite", "Chantilly", "Coco ralado", "Canela", "Fermento biologico", 
    "Linguica", "Carne moida", "Costela", "Bacon", "Hamburguer", "Lasanha", "Pizza", "Pao de queijo", "Tapioca", "Geleia", "Mel", "Pasta de amendoim", "Sardinha", "Atum", "Sopa instantanea", "Batata", "Cebola", "Tomate", 
    "Alho", "Cenoura", "Abobora", "Chuchu", "Pimentao", "Alface", "Repolho","Couve", "Brocolis", "Banana", "Maca", "Laranja", "Limao", "Mamao", "Abacaxi", "Melancia", "Melao", "Uva", "Morango", "Kiwi", "Pera", "Manga", "Goiaba", "Caju", 
    "Maracuja", "Acerola", "Coco", "Pessego", "Ameixa", "Figo", "Caqui", "Jabuticaba", "Cupuacu", "Acai", "Guarana", "Pinhao", "Palmito", "Azeitona", "Pepino", "Rabanete", "Beterraba", "Inhame", "Mandioca", "Batata-doce", "Gengibre", 
    "Coentro", "Salsinha", "Farinha de rosca", "Fermento quimico", "Cacau em po", "Cha mate", "Cha verde", "Cha de camomila", "Cha de hortela", "Cafe soluvel", "Cafe em capsula", "Cafe descafeinado", "Leite em po", "Leite de soja", 
    "Leite de amendoas", "Leite de coco", "Leite de aveia", "Leite sem lactose", "Iogurte grego", "Iogurte natural", "Iogurte com frutas", "Iogurte zero lactose", "Queijo minas", "Queijo prato", "Queijo mussarela", "Queijo parmesao", 
    "Queijo provolone", "Queijo cottage", "Queijo ricota", "Queijo gorgonzola", "Queijo brie", "Queijo coalho", "Presunto cozido", "Presunto defumado", "Peito de peru", "Salame", "Mortadela", "Rosbife", "Carne seca", "Charque", 
    "Bacalhau", "Tilapia", "Salmao", "Camarao", "Lula", "Polvo", "Ostra", "Mexilhao", "Sushi", "Temaki", "Yakult", "Danoninho", "Biscoito recheado", "Biscoito agua e sal", "Biscoito cream cracker", "Biscoito de polvilho", "Biscoito de aveia",
    "Biscoito de gengibre", "Biscoito de chocolate", "Biscoito de coco", "Biscoito de maisena", "Biscoito de nata", "Pao integral", "Pao frances", "Pao de forma", "Pao sirio", "Pao de centeio", "Pao de aveia", "Pao de milho", "Pao de batata", 
    "Pao de queijo congelado", "Pao de alho", "Salsicha hot dog", "Salsicha para churrasco", "Salsicha de frango", "Salsicha de peru", "Salsicha vegetariana", "Hamburguer de frango", "Hamburguer vegetariano", "Hamburguer de soja", 
    "Hamburguer de peixe", "Hamburguer artesanal", "Pipoca doce", "Pipoca salgada", "Pipoca de micro-ondas", "Pipoca caramelizada", "Pipoca com manteiga", "Pipoca com queijo", "Pipoca com bacon", "Pipoca colorida", "Pipoca zero acucar", 
    "Pipoca zero gordura","Pizza congelada", "Lasanha congelada", "Pao de queijo congelado", "Hamburguer congelado", "Batata frita congelada", "Nuggets de frango", "Empanado de frango", "Empanado de peixe", "Escondidinho congelado", "Feijoada congelada",
    "Risoto congelado", "Yakisoba congelado", "Coxinha congelada", "Kibe congelado", "Pastel congelado", "Torta congelada", "Pao de alho congelado", "Pao de queijo recheado congelado", "Sorvete de massa", "Sorvete de palito",
    "Acai congelado", "Polenta congelada", "Legumes congelados", "Frutas congeladas", "Pure de batata congelado", "Pao frances congelado", "Pao de forma congelado", "Pao integral congelado", "Pao sirio congelado", "Pao de centeio congelado",
    "Vodka", "Whisky", "Rum", "Gin", "Tequila", "Cachaca", "Licor", "Champagne", "Espumante", "Sidra", "Conhaque", "Vermute", "Sake", "Absinto", "Catuaba", "Pinga artesanal", "Vinho tinto", "Vinho branco", "Vinho rose", "Vinho seco", 
    "Vinho suave", "Cerveja artesanal", "Cerveja sem alcool", "Cerveja de trigo", "Cerveja IPA", "Cerveja stout", "Cerveja pilsen", "Cerveja light", "Cerveja zero alcool", "Cerveja de milho", 
    "Oleo de girassol", "Oleo de milho", "Oleo de canola", "Oleo de coco", "Oleo de linhaca", "Oleo de gergelim", "Oleo de amendoim", "Oleo de algodao", "Oleo de palma", "Oleo de abacate",
    "Arroz integral", "Arroz arboreo", "Arroz negro", "Arroz vermelho", "Arroz cateto", "Arroz jasmine", "Arroz basmati", "Arroz parboilizado", "Arroz selvagem", "Arroz 7 graos",
    "Farinha de milho", "Farinha de aveia", "Farinha de centeio", "Farinha de cevada", "Farinha de coco", "Farinha de amendoas", "Farinha de arroz", "Farinha de grao-de-bico", "Farinha de quinoa", "Farinha de linhaca",
    "Farinha de tapioca", "Farinha de banana verde", "Farinha de maracuja", "Farinha de berinjela", "Farinha de soja", "Farinha de trigo integral", "Farinha de trigo sarraceno", "Farinha de trigo sem gluten", "Farinha de trigo especial", "Farinha de trigo tipo 1",
    "Desodorante roll-on", "Desodorante aerosol", "Sabonete liquido", "Sabonete em barra", "Creme para barbear", "Locao pos-barba", "Protetor solar", "Hidratante corporal", "Lenco umedecido", "Absorvente noturno",
    "Absorvente diario", "Cotonete", "Fio dental", "Enxaguante bucal", "Pasta de dente clareadora", "Escova de dentes infantil", "Escova de dentes eletrica", "Creme para pentear", "Gel para cabelo", "Mascara capilar",
    "Condicionador reparador", "Shampoo anticaspa", "Shampoo infantil", "Sabonete intimo", "Creme para assaduras", "Oleo para bebe", "Lenco de papel", "Papel higienico perfumado", "Papel higienico biodegradavel", "Sabonete antibacteriano",
    "Desinfetante para as maos", "Creme para os pes", "Esmalte de unha", "Removedor de esmalte", "Cera depilatoria", "Lamina de barbear descartavel", "Creme para rugas", "Protetor labial", "Po compacto", "Batom",
    "Base liquida", "Demaquilante", "Creme para olheiras", "Creme para acne", "Protetor solar facial", "Shampoo seco", "Condicionador sem enxague", "Gel para sobrancelha", "Creme para maos", "Sabonete esfoliante",
    "Detergente neutro", "Detergente concentrado", "Amaciante perfumado", "Amaciante hipoalergenico", "Desinfetante para banheiro", "Desinfetante para cozinha", "Limpador multiuso", "Limpador para vidros", "Limpador para pisos", "Lustra-moveis",
    "Saco de lixo", "Saco de lixo reciclavel", "Saco de lixo biodegradavel", "Vassoura", "Rodo", "Pa de lixo", "Balde", "Esponja de aco", "Esponja multiuso", "Luvas de limpeza",
    "Alcool em gel", "Alcool liquido", "Limpa-vidros", "Limpa-forno", "Limpa-carpete", "Limpa-azulejo", "Limpa-inox", "Limpa-pisos", "Limpa-tapetes", "Limpa-estofados",
    "Desengordurante", "Removedor de manchas", "Cera para pisos", "Cera para moveis", "Cera para carro", "Inseticida", "Repelente de insetos", "Aromatizador de ambientes", "Difusor de aromas", "Velas aromaticas",
    "Panela de pressao", "Panela antiaderente", "Panela de ferro", "Panela de barro", "Frigideira", "Chaleira", "Liquidificador", "Batedeira", "Processador de alimentos", "Torradeira",
    "Forno eletrico", "Air fryer", "Cafeteira", "Jarra eletrica", "Faca de cozinha", "Tabua de corte", "Escorredor de louca", "Espatula de silicone", "Colher de pau", "Ralador de queijo",
    "Descascador de legumes", "Assadeira", "Forma de bolo", "Forma de gelo", "Garrafa termica", "Caneca", "Prato", "Tigela", "Talheres", "Guardanapo de pano",
    "Toalha de mesa", "Jogo americano", "Vaso decorativo", "Porta-temperos", "Porta-sabao", "Escova de limpeza", "Rodo de pia", "Vassoura de pia", "Organizador de gavetas", "Cabide",
    "Caderno", "Caneta", "Lapis", "Borracha", "Apontador", "Regua", "Estojo", "Mochila", "Cola branca", "Tesoura"
]

mercearia = [
    "Arroz", "Feijao", "Acucar", "Sal", "Oleo de soja", "Cafe", "Farinha de trigo", "Macarrao", "Molho de tomate", "Farinha de mandioca", "Polenta", "Lentilha", "Grao-de-bico", "Soja", "Aveia", "Granola", "Barra de cereal", "Pipoca", "Amendoim", "Castanha",
    "Nozes", "Uva passa", "Azeite", "Vinagre", "Fermento em po", "Gelatina", "Pudim", "Leite condensado", "Creme de leite", "Chantilly","Coco ralado", "Canela", "Fermento biologico", "Tapioca", "Geleia", "Mel", "Pasta de amendoim", "Sopa instantanea", 
    "Filtro de cafe", "Cereal matinal", "Achocolatado", "Extrato de tomate", "Maionese", "Mostarda", "Ketchup", "Tempero pronto", "Caldo de carne", "Caldo de galinha", "Seleta de legumes", "Farinha de rosca",
    "Fermento quimico", "Cacau em po", "Cha mate", "Cha verde", "Cha de camomila", "Cha de hortela", "Cafe soluvel", "Cafe em capsula", "Cafe descafeinado", "Oleo de girassol",
    "Oleo de milho", "Oleo de canola", "Oleo de coco", "Oleo de linhaca", "Oleo de gergelim", "Oleo de amendoim", "Oleo de algodao", "Oleo de palma", "Oleo de abacate", "Arroz integral",
    "Arroz arboreo", "Arroz negro", "Arroz vermelho", "Arroz cateto", "Arroz jasmine", "Arroz basmati", "Arroz parboilizado", "Arroz selvagem", "Arroz 7 graos", "Farinha de milho",
    "Farinha de aveia", "Farinha de centeio", "Farinha de cevada", "Farinha de coco", "Farinha de amendoas", "Farinha de arroz", "Farinha de grao-de-bico", "Farinha de quinoa", "Farinha de linhaca", "Farinha de tapioca",
    "Farinha de banana verde", "Farinha de maracuja", "Farinha de berinjela", "Farinha de soja", "Farinha de trigo integral", "Farinha de trigo sarraceno", "Farinha de trigo sem gluten", "Farinha de trigo especial", "Farinha de trigo tipo 1",
    "Salgadinho", "Chocolate", "Balas", "Goma de mascar", "Pipoca doce", "Pipoca salgada", "Pipoca de micro-ondas", "Pipoca caramelizada", "Pipoca com manteiga", "Pipoca com queijo","Pipoca com bacon", "Pipoca colorida", "Pipoca zero acucar", 
    "Pipoca zero gordura", "Sopa instantanea", "Farinha de trigo tipo 1", "Filtro de cafe", "Cereal matinal", "Achocolatado", "Leite em po", "Leite de soja", "Leite de amendoas", "Leite de coco", "Leite de aveia", 
    "Leite sem lactose", "Biscoito recheado", "Biscoito agua e sal", "Biscoito cream cracker", "Biscoito de polvilho", "Biscoito de aveia", "Biscoito de gengibre", "Biscoito de chocolate", "Biscoito de coco", "Biscoito de maisena", "Biscoito de nata"
]

produtos_limpeza = [
    "Detergente", "Amaciante", "Agua sanitaria", "Desinfetante", "Esponja de lavar louca", "Sacola plastica", "Guardanapo", "Toalha de papel", "Sabao em po", "Limpador multiuso",
    "Detergente neutro", "Detergente concentrado", "Amaciante perfumado", "Amaciante hipoalergenico", "Desinfetante para banheiro", "Desinfetante para cozinha", "Limpador multiuso", "Limpador para vidros", "Limpador para pisos", "Lustra-moveis",
    "Saco de lixo", "Saco de lixo reciclavel", "Saco de lixo biodegradavel", "Vassoura", "Rodo", "Pa de lixo", "Balde", "Esponja de aco", "Esponja multiuso", "Luvas de limpeza",
    "Alcool em gel", "Alcool liquido", "Limpa-vidros", "Limpa-forno", "Limpa-carpete", "Limpa-azulejo", "Limpa-inox", "Limpa-pisos", "Limpa-tapetes", "Limpa-estofados",
    "Desengordurante", "Removedor de manchas", "Cera para pisos", "Cera para moveis", "Cera para carro", "Inseticida", "Repelente de insetos", "Aromatizador de ambientes", "Difusor de aromas", "Velas aromaticas"
]

higiene_pessoal = [
    "Sabonete", "Shampoo", "Condicionador", "Papel higienico", "Creme dental", "Escova de dentes", "Desodorante", "Absorvente", "Lamina de barbear", "Creme hidratante",
    "Desodorante roll-on", "Desodorante aerosol", "Sabonete liquido", "Sabonete em barra", "Creme para barbear", "Locao pos-barba", "Protetor solar", "Hidratante corporal", "Lenco umedecido", "Absorvente noturno",
    "Absorvente diario", "Cotonete", "Fio dental", "Enxaguante bucal", "Pasta de dente clareadora", "Escova de dentes infantil", "Escova de dentes eletrica", "Creme para pentear", "Gel para cabelo", "Mascara capilar",
    "Condicionador reparador", "Shampoo anticaspa", "Shampoo infantil", "Sabonete intimo", "Creme para assaduras", "Oleo para bebe", "Lenco de papel", "Papel higienico perfumado", "Papel higienico biodegradavel", "Sabonete antibacteriano",
    "Desinfetante para as maos", "Creme para os pes", "Esmalte de unha", "Removedor de esmalte", "Cera depilatoria", "Lamina de barbear descartavel", "Creme para rugas", "Protetor labial", "Po compacto", "Batom",
    "Base liquida", "Demaquilante", "Creme para olheiras", "Creme para acne", "Protetor solar facial", "Shampoo seco", "Condicionador sem enxague", "Gel para sobrancelha", "Creme para maos", "Sabonete esfoliante"
]

bebidas_nao_alcoolicas = [
    "Refrigerante", "Suco", "Agua mineral", "Cha", "Achocolatado", "Cafe", "Leite", "Energetico", "Cerveja sem alcool", "Cerveja zero alcool"
]

bebidas_alcoolicas = [
    "Cerveja", "Vinho", "Vodka", "Whisky", "Rum", "Gin", "Tequila", "Cachaca", "Licor", "Champagne",
    "Espumante", "Sidra", "Conhaque", "Vermute", "Sake", "Absinto", "Catuaba", "Pinga artesanal", "Vinho tinto", "Vinho branco",
    "Vinho rose", "Vinho seco", "Vinho suave", "Cerveja artesanal", "Cerveja de trigo", "Cerveja IPA", "Cerveja stout", "Cerveja pilsen", "Cerveja light", "Cerveja de milho"
]

laticinios = [
    "Leite", "Queijo", "Manteiga", "Margarina", "Iogurte", "Requeijao", "Creme de leite", "Leite condensado", "Chantilly", "Fermento biologico", "Leite em po", "Leite de soja", "Leite de amendoas", "Leite de coco","Leite de aveia", "Leite sem lactose", 
    "Iogurte grego", "Iogurte natural", "Iogurte com frutas", "Iogurte zero lactose", "Queijo minas", "Queijo prato", "Queijo mussarela", "Queijo parmesao", "Queijo provolone", "Queijo cottage", "Queijo ricota", "Queijo gorgonzola", "Queijo brie", 
    "Queijo coalho", "Yakult", "Danoninho", "Iogurte grego", "Iogurte natural", "Iogurte com frutas", "Iogurte zero lactose", "Queijo minas", "Queijo prato", "Queijo mussarela", "Queijo parmesao", "Queijo provolone", "Queijo cottage", "Queijo ricota", "Queijo gorgonzola", 
    "Queijo brie", "Queijo coalho"
]

carnes = [
    "Carne bovina", "Frango", "Peixe", "Salsicha", "Presunto", "Linguica", "Carne moida", "Costela", "Bacon", "Hamburguer", "Presunto cozido", "Presunto defumado", "Peito de peru", "Salame", "Mortadela", "Rosbife", "Carne seca", "Charque", "Bacalhau", "Tilapia",
    "Salmao", "Camarao", "Lula", "Polvo", "Ostra", "Mexilhao", "Sushi", "Temaki", "Salsicha hot dog", "Salsicha para churrasco", "Salsicha de frango", "Salsicha de peru", "Salsicha vegetariana", "Hamburguer de frango", "Hamburguer vegetariano",
    "Hamburguer de soja", "Hamburguer de peixe", "Hamburguer artesanal", "Presunto cozido", "Presunto defumado", "Peito de peru", "Salame", "Mortadela", "Rosbife", "Carne seca", "Charque", "Bacalhau", "Tilapia", "Salmao", "Camarao", "Lula", "Polvo", 
    "Ostra", "Mexilhao", "Sushi", "Temaki", "Salsicha hot dog", "Salsicha para churrasco", "Salsicha de frango", "Salsicha de peru", "Salsicha vegetariana", "Hamburguer de frango", "Hamburguer vegetariano", "Hamburguer de soja", "Hamburguer de peixe", 
    "Hamburguer artesanal"
]

congelados = [
    "Pizza congelada", "Lasanha congelada", "Pao de queijo congelado", "Hamburguer congelado", "Batata frita congelada", "Nuggets de frango", "Empanado de frango", "Empanado de peixe", "Escondidinho congelado", "Feijoada congelada", 
    "Pao de queijo congelado", "Risoto congelado", "Yakisoba congelado", "Coxinha congelada", "Kibe congelado", "Pastel congelado", "Torta congelada", "Pao de alho congelado", "Pao de queijo recheado congelado", "Sorvete de massa", 
    "Sorvete de palito", "Sorvete","Acai congelado", "Polenta congelada", "Legumes congelados", "Frutas congeladas", "Pure de batata congelado", "Pao frances congelado", "Pao de forma congelado", "Pao integral congelado", 
    "Pao sirio congelado", "Pao de centeio congelado"
]

enlatados = [
    "Milho", "Ervilha", "Seleta de legumes", "Sardinha", "Atum", "Molho de tomate", "Extrato de tomate", "Palmito", "Azeitona", "Seleta de legumes"
]

hortifruti = [
    "Batata", "Cebola", "Tomate", "Alho", "Cenoura", "Abobora", "Chuchu", "Pimentao", "Alface", "Repolho",
    "Couve", "Brocolis", "Banana", "Maca", "Laranja", "Limao", "Mamao", "Abacaxi", "Melancia", "Melao",
    "Uva", "Morango", "Kiwi", "Pera", "Manga", "Goiaba", "Caju", "Maracuja", "Acerola", "Coco",
    "Pessego", "Ameixa", "Figo", "Caqui", "Jabuticaba", "Cupuacu", "Acai", "Guarana", "Pinhao", "Pepino",
    "Rabanete", "Beterraba", "Inhame", "Mandioca", "Batata-doce", "Gengibre", "Coentro", "Salsinha"
]

padaria = [
    "Pao", "Biscoito", "Bolacha", "Pao de queijo", "Pizza", "Lasanha", "Pao integral", "Pao frances", "Pao de forma", "Pao sirio",
    "Pao de centeio", "Pao de aveia", "Pao de milho", "Pao de batata", "Pao de queijo congelado", "Pao de alho", "Biscoito recheado", "Biscoito agua e sal", "Biscoito cream cracker", "Biscoito de polvilho",
    "Biscoito de aveia", "Biscoito de gengibre", "Biscoito de chocolate", "Biscoito de coco", "Biscoito de maisena", "Biscoito de nata", "Pao integral", "Pao frances", "Pao de forma", "Pao sirio", 
    "Pao de centeio", "Pao de aveia", "Pao de milho", "Pao de batata", "Pao de queijo congelado", "Pao de alho"
]

utensilios = [
    "Panela de pressao", "Panela antiaderente", "Panela de ferro", "Panela de barro", "Frigideira", "Chaleira", "Liquidificador", "Batedeira", "Processador de alimentos", "Torradeira",
    "Forno eletrico", "Air fryer", "Cafeteira", "Jarra eletrica", "Faca de cozinha", "Tabua de corte", "Escorredor de louca", "Espatula de silicone", "Colher de pau", "Ralador de queijo",
    "Descascador de legumes", "Assadeira", "Forma de bolo", "Forma de gelo", "Garrafa termica", "Caneca", "Prato", "Tigela", "Talheres", "Guardanapo de pano",
    "Toalha de mesa", "Jogo americano", "Vaso decorativo", "Porta-temperos", "Porta-sabao", "Escova de limpeza", "Rodo de pia", "Vassoura de pia", "Organizador de gavetas", "Cabide",
    "Caderno", "Caneta", "Lapis", "Borracha", "Apontador", "Regua", "Estojo", "Mochila", "Cola branca", "Tesoura"
]

# Função para gerar um produto aleatorio e o id da sua categoria

def gerar_produto_categoria():
    produto = random.choice(produtos)
    if produto in mercearia:
        id_categoria = 1
    elif produto in produtos_limpeza:
        id_categoria = 2
    elif produto in  higiene_pessoal:
        id_categoria = 3   
    elif produto in  bebidas_nao_alcoolicas:
        id_categoria = 4
    elif produto in  bebidas_alcoolicas:
        id_categoria = 5
    elif produto in  laticinios:
        id_categoria = 6
    elif produto in  carnes:
        id_categoria = 7
    elif produto in  congelados:
        id_categoria = 8
    elif produto in  enlatados:
        id_categoria = 9    
    elif produto in  hortifruti:
        id_categoria = 10 
    elif produto in  padaria:
        id_categoria = 11 
    else: 
        id_categoria = 12
    
    return produto, id_categoria

dados_produto = [gerar_produto_categoria() for _ in range(10000)]

list_apenas_produtos = [item[0] for item in dados_produto]

#print(type(dados_produto))
#print(list_apenas_produtos)

df_produto = pd.DataFrame(dados_produto)

#df_produto.head(20)


In [ ]:
#Script para gerar os valores dos produtos

import random
import pandas as pd

# Lista de produtos da mercearia
#mercearia = [
#    "Arroz", "Feijao", "Acucar", "Sal", "Oleo de soja", "Cafe", "Farinha de trigo", "Macarrao", "Molho de tomate", "Farinha de mandioca", 
#    "Polenta", "Lentilha", "Grao-de-bico", "Soja", "Aveia", "Granola", "Barra de cereal", "Pipoca", "Amendoim", "Castanha",
#    "Nozes", "Uva passa", "Azeite", "Vinagre", "Fermento em po", "Gelatina", "Pudim", "Leite condensado", "Creme de leite", "Chantilly",
#    "Coco ralado", "Canela", "Fermento biologico", "Tapioca", "Geleia", "Mel", "Pasta de amendoim", "Sopa instantanea", 
#    "Filtro de cafe", "Cereal matinal", "Achocolatado", "Extrato de tomate", "Maionese", "Mostarda", "Ketchup", "Tempero pronto", 
#    "Caldo de carne", "Caldo de galinha", "Seleta de legumes", "Farinha de rosca", "Fermento quimico", "Cacau em po", "Cha mate", 
#    "Cha verde", "Cha de camomila", "Cha de hortela", "Cafe soluvel", "Cafe em capsula", "Cafe descafeinado", "Oleo de girassol",
#    "Oleo de milho", "Oleo de canola", "Oleo de coco", "Oleo de linhaca", "Oleo de gergelim", "Oleo de amendoim", "Oleo de algodao", 
#    "Oleo de palma", "Oleo de abacate", "Arroz integral", "Arroz arboreo", "Arroz negro", "Arroz vermelho", "Arroz cateto", 
#    "Arroz jasmine", "Arroz basmati", "Arroz parboilizado", "Arroz selvagem", "Arroz 7 graos", "Farinha de milho", "Farinha de aveia", 
#    "Farinha de centeio", "Farinha de cevada", "Farinha de coco", "Farinha de amendoas", "Farinha de arroz", "Farinha de grao-de-bico", 
#    "Farinha de quinoa", "Farinha de linhaca", "Farinha de tapioca", "Farinha de banana verde", "Farinha de maracuja", 
#    "Farinha de berinjela", "Farinha de soja", "Farinha de trigo integral", "Farinha de trigo sarraceno", "Farinha de trigo sem gluten", 
#    "Farinha de trigo especial", "Farinha de trigo tipo 1", "Salgadinho", "Chocolate", "Balas", "Goma de mascar", "Pipoca doce", 
#    "Pipoca salgada", "Pipoca de micro-ondas", "Pipoca caramelizada", "Pipoca com manteiga", "Pipoca com queijo", "Pipoca com bacon", 
#    "Pipoca colorida", "Pipoca zero acucar", "Pipoca zero gordura", "Sopa instantanea", "Farinha de trigo tipo 1", "Filtro de cafe", 
#    "Cereal matinal", "Achocolatado", "Leite em po", "Leite de soja", "Leite de amendoas", "Leite de coco", "Leite de aveia", 
#    "Leite sem lactose", "Biscoito recheado", "Biscoito agua e sal", "Biscoito cream cracker", "Biscoito de polvilho", 
#    "Biscoito de aveia", "Biscoito de gengibre", "Biscoito de chocolate", "Biscoito de coco", "Biscoito de maisena", "Biscoito de nata"
#]

#mercearia = ["Creme para rugas"]

# Intervalos de preços (em reais) para cada produto, baseados em valores de mercado no Brasil
intervalos_precos = {
    "Ovos": (8.00,13.00),            # preco por duzia
    "Arroz": (5.00, 10.00),          # Preço por kg
    "Feijao": (8.00, 15.00),         # Preço por kg
    "Acucar": (3.50, 6.00),          # Preço por kg
    "Sal": (2.00, 4.00),             # Preço por kg
    "Oleo de soja": (7.00, 12.00),   # Preço por litro
    "Cafe": (15.00, 30.00),          # Preço por kg
    "Farinha de trigo": (4.00, 8.00), # Preço por kg
    "Macarrao": (3.00, 6.00),        # Preço por 500g
    "Molho de tomate": (2.50, 5.00), # Preço por lata
    "Farinha de mandioca": (5.00, 10.00), # Preço por kg
    "Polenta": (4.00, 8.00),         # Preço por kg
    "Lentilha": (6.00, 12.00),       # Preço por kg
    "Grao-de-bico": (7.00, 14.00),   # Preço por kg
    "Soja": (5.00, 10.00),           # Preço por kg
    "Aveia": (6.00, 12.00),          # Preço por kg
    "Granola": (10.00, 20.00),       # Preço por kg
    "Barra de cereal": (1.50, 3.00), # Preço por unidade
    "Pipoca": (2.00, 5.00),          # Preço por pacote
    "Amendoim": (8.00, 15.00),       # Preço por kg
    "Castanha": (20.00, 40.00),      # Preço por kg
    "Nozes": (30.00, 60.00),         # Preço por kg
    "Uva passa": (10.00, 20.00),     # Preço por kg
    "Azeite": (15.00, 30.00),        # Preço por litro
    "Vinagre": (3.00, 6.00),         # Preço por litro
    "Fermento em po": (2.00, 4.00),  # Preço por pacote
    "Gelatina": (1.50, 3.00),        # Preço por caixa
    "Pudim": (5.00, 10.00),          # Preço por unidade
    "Leite condensado": (4.00, 8.00), # Preço por lata
    "Creme de leite": (3.00, 6.00),  # Preço por caixa
    "Chantilly": (5.00, 10.00),      # Preço por lata
    "Coco ralado": (4.00, 8.00),     # Preço por pacote
    "Canela": (3.00, 6.00),          # Preço por pacote
    "Fermento biologico": (2.00, 4.00), # Preço por unidade
    "Tapioca": (5.00, 10.00),        # Preço por kg
    "Geleia": (4.00, 8.00),          # Preço por pote
    "Mel": (10.00, 20.00),           # Preço por kg
    "Pasta de amendoim": (8.00, 15.00), # Preço por pote
    "Sopa instantanea": (2.00, 5.00), # Preço por unidade
    "Filtro de cafe": (5.00, 10.00), # Preço por pacote
    "Cereal matinal": (8.00, 15.00), # Preço por caixa
    "Achocolatado": (6.00, 12.00),   # Preço por kg
    "Extrato de tomate": (2.50, 5.00), # Preço por lata
    "Maionese": (4.00, 8.00),        # Preço por pote
    "Mostarda": (3.00, 6.00),        # Preço por pote
    "Ketchup": (3.00, 6.00),         # Preço por pote
    "Tempero pronto": (2.00, 4.00),  # Preço por pacote
    "Caldo de carne": (1.50, 3.00),  # Preço por tablete
    "Caldo de galinha": (1.50, 3.00), # Preço por tablete
    "Seleta de legumes": (3.00, 6.00), # Preço por lata
    "Farinha de rosca": (3.00, 6.00), # Preço por pacote
    "Fermento quimico": (2.00, 4.00), # Preço por pacote
    "Cacau em po": (6.00, 12.00),    # Preço por kg
    "Cha mate": (5.00, 10.00),       # Preço por pacote
    "Cha verde": (5.00, 10.00),      # Preço por pacote
    "Cha de camomila": (5.00, 10.00), # Preço por pacote
    "Cha de hortela": (5.00, 10.00), # Preço por pacote
    "Cafe soluvel": (10.00, 20.00),  # Preço por pote
    "Cafe em capsula": (15.00, 30.00), # Preço por caixa
    "Cafe descafeinado": (10.00, 20.00), # Preço por pote
    "Oleo de girassol": (7.00, 12.00), # Preço por litro
    "Oleo de milho": (7.00, 12.00),  # Preço por litro
    "Oleo de canola": (7.00, 12.00), # Preço por litro
    "Oleo de coco": (10.00, 20.00),  # Preço por litro
    "Oleo de linhaca": (15.00, 30.00), # Preço por litro
    "Oleo de gergelim": (15.00, 30.00), # Preço por litro
    "Oleo de amendoim": (10.00, 20.00), # Preço por litro
    "Oleo de algodao": (7.00, 12.00), # Preço por litro
    "Oleo de palma": (7.00, 12.00),  # Preço por litro
    "Oleo de abacate": (15.00, 30.00), # Preço por litro
    "Arroz integral": (6.00, 12.00), # Preço por kg
    "Arroz arboreo": (8.00, 15.00),  # Preço por kg
    "Arroz negro": (10.00, 20.00),   # Preço por kg
    "Arroz vermelho": (10.00, 20.00), # Preço por kg
    "Arroz cateto": (8.00, 15.00),   # Preço por kg
    "Arroz jasmine": (10.00, 20.00), # Preço por kg
    "Arroz basmati": (12.00, 25.00), # Preço por kg
    "Arroz parboilizado": (6.00, 12.00), # Preço por kg
    "Arroz selvagem": (15.00, 30.00), # Preço por kg
    "Arroz 7 graos": (10.00, 20.00), # Preço por kg
    "Farinha de milho": (4.00, 8.00), # Preço por kg
    "Farinha de aveia": (6.00, 12.00), # Preço por kg
    "Farinha de centeio": (6.00, 12.00), # Preço por kg
    "Farinha de cevada": (6.00, 12.00), # Preço por kg
    "Farinha de coco": (8.00, 15.00), # Preço por kg
    "Farinha de amendoas": (15.00, 30.00), # Preço por kg
    "Farinha de arroz": (6.00, 12.00), # Preço por kg
    "Farinha de grao-de-bico": (7.00, 14.00), # Preço por kg
    "Farinha de quinoa": (10.00, 20.00), # Preço por kg
    "Farinha de linhaca": (10.00, 20.00), # Preço por kg
    "Farinha de tapioca": (5.00, 10.00), # Preço por kg
    "Farinha de banana verde": (8.00, 15.00), # Preço por kg
    "Farinha de maracuja": (10.00, 20.00), # Preço por kg
    "Farinha de berinjela": (8.00, 15.00), # Preço por kg
    "Farinha de soja": (6.00, 12.00), # Preço por kg
    "Farinha de trigo integral": (5.00, 10.00), # Preço por kg
    "Farinha de trigo sarraceno": (8.00, 15.00), # Preço por kg
    "Farinha de trigo sem gluten": (10.00, 20.00), # Preço por kg
    "Farinha de trigo especial": (6.00, 12.00), # Preço por kg
    "Farinha de trigo tipo 1": (4.00, 8.00), # Preço por kg
    "Salgadinho": (2.00, 5.00),      # Preço por pacote
    "Chocolate": (5.00, 10.00),      # Preço por barra
    "Balas": (1.00, 3.00),           # Preço por pacote
    "Goma de mascar": (2.00, 5.00),  # Preço por pacote
    "Pipoca doce": (3.00, 6.00),     # Preço por pacote
    "Pipoca salgada": (3.00, 6.00),  # Preço por pacote
    "Pipoca de micro-ondas": (4.00, 8.00), # Preço por pacote
    "Pipoca caramelizada": (5.00, 10.00), # Preço por pacote
    "Pipoca com manteiga": (5.00, 10.00), # Preço por pacote
    "Pipoca com queijo": (5.00, 10.00), # Preço por pacote
    "Pipoca com bacon": (5.00, 10.00), # Preço por pacote
    "Pipoca colorida": (5.00, 10.00), # Preço por pacote
    "Pipoca zero acucar": (5.00, 10.00), # Preço por pacote
    "Pipoca zero gordura": (5.00, 10.00), # Preço por pacote
    "Sopa instantanea": (2.00, 5.00), # Preço por unidade
    "Farinha de trigo tipo 1": (4.00, 8.00), # Preço por kg
    "Filtro de cafe": (5.00, 10.00), # Preço por pacote
    "Cereal matinal": (8.00, 15.00), # Preço por caixa
    "Achocolatado": (6.00, 12.00),   # Preço por kg
    "Leite em po": (10.00, 20.00),   # Preço por lata
    "Leite de soja": (5.00, 10.00),  # Preço por litro
    "Leite de amendoas": (8.00, 15.00), # Preço por litro
    "Leite de coco": (6.00, 12.00),  # Preço por litro
    "Leite de aveia": (6.00, 12.00), # Preço por litro
    "Leite sem lactose": (5.00, 10.00), # Preço por litro
    "Biscoito recheado": (2.00, 5.00), # Preço por pacote
    "Biscoito agua e sal": (2.00, 5.00), # Preço por pacote
    "Biscoito cream cracker": (2.00, 5.00), # Preço por pacote
    "Biscoito de polvilho": (3.00, 6.00), # Preço por pacote
    "Biscoito de aveia": (3.00, 6.00), # Preço por pacote
    "Biscoito de gengibre": (3.00, 6.00), # Preço por pacote
    "Biscoito de chocolate": (3.00, 6.00), # Preço por pacote
    "Biscoito de coco": (3.00, 6.00), # Preço por pacote
    "Biscoito de maisena": (3.00, 6.00), # Preço por pacote
    "Biscoito de nata": (3.00, 6.00),  # Preço por pacote
    "Detergente": (2.00, 5.00),                  # Preço por litro
    "Amaciante": (8.00, 15.00),                  # Preço por litro
    "Agua sanitaria": (3.00, 6.00),              # Preço por litro
    "Desinfetante": (5.00, 10.00),               # Preço por litro
    "Esponja de lavar louca": (1.50, 3.00),      # Preço por unidade
    "Sacola plastica": (0.50, 1.50),             # Preço por pacote
    "Guardanapo": (2.00, 4.00),                  # Preço por pacote
    "Toalha de papel": (3.00, 6.00),             # Preço por rolo
    "Sabao em po": (8.00, 15.00),                # Preço por kg
    "Limpador multiuso": (5.00, 10.00),          # Preço por litro
    "Detergente neutro": (2.00, 5.00),           # Preço por litro
    "Detergente concentrado": (3.00, 6.00),      # Preço por litro
    "Amaciante perfumado": (8.00, 15.00),        # Preço por litro
    "Amaciante hipoalergenico": (10.00, 20.00),  # Preço por litro
    "Desinfetante para banheiro": (5.00, 10.00), # Preço por litro
    "Desinfetante para cozinha": (5.00, 10.00),  # Preço por litro
    "Limpador para vidros": (5.00, 10.00),       # Preço por litro
    "Limpador para pisos": (5.00, 10.00),        # Preço por litro
    "Lustra-moveis": (5.00, 10.00),              # Preço por litro
    "Saco de lixo": (5.00, 10.00),               # Preço por pacote
    "Saco de lixo reciclavel": (6.00, 12.00),    # Preço por pacote
    "Saco de lixo biodegradavel": (7.00, 14.00), # Preço por pacote
    "Vassoura": (10.00, 20.00),                  # Preço por unidade
    "Rodo": (5.00, 10.00),                       # Preço por unidade
    "Pa de lixo": (5.00, 10.00),                 # Preço por unidade
    "Balde": (10.00, 20.00),                     # Preço por unidade
    "Esponja de aco": (2.00, 4.00),              # Preço por unidade
    "Esponja multiuso": (3.00, 6.00),            # Preço por unidade
    "Luvas de limpeza": (5.00, 10.00),           # Preço por par
    "Alcool em gel": (10.00, 20.00),             # Preço por litro
    "Alcool liquido": (8.00, 15.00),             # Preço por litro
    "Limpa-vidros": (5.00, 10.00),               # Preço por litro
    "Limpa-forno": (5.00, 10.00),                # Preço por litro
    "Limpa-carpete": (5.00, 10.00),              # Preço por litro
    "Limpa-azulejo": (5.00, 10.00),              # Preço por litro
    "Limpa-inox": (5.00, 10.00),                 # Preço por litro
    "Limpa-pisos": (5.00, 10.00),                # Preço por litro
    "Limpa-tapetes": (5.00, 10.00),              # Preço por litro
    "Limpa-estofados": (5.00, 10.00),            # Preço por litro
    "Desengordurante": (5.00, 10.00),            # Preço por litro
    "Removedor de manchas": (5.00, 10.00),       # Preço por litro
    "Cera para pisos": (10.00, 20.00),           # Preço por litro
    "Cera para moveis": (10.00, 20.00),          # Preço por litro
    "Cera para carro": (10.00, 20.00),           # Preço por litro
    "Inseticida": (5.00, 10.00),                 # Preço por litro
    "Repelente de insetos": (5.00, 10.00),       # Preço por litro
    "Aromatizador de ambientes": (5.00, 10.00),  # Preço por unidade
    "Difusor de aromas": (10.00, 20.00),         # Preço por unidade
    "Velas aromaticas": (5.00, 10.00),            # Preço por unidade
    "Sabonete": (2.50, 10.00),
    "Shampoo": (10.00, 40.00),
    "Condicionador": (10.00, 40.00),
    "Papel higienico": (5.00, 25.00),
    "Creme dental": (3.50, 15.00),
    "Escova de dentes": (5.00, 20.00),
    "Desodorante": (7.00, 25.00),
    "Absorvente": (5.00, 20.00),
    "Lamina de barbear": (5.00, 30.00),
    "Creme hidratante": (10.00, 50.00),
    "Desodorante roll-on": (8.00, 18.00),
    "Desodorante aerosol": (10.00, 30.00),
    "Sabonete liquido": (8.00, 25.00),
    "Sabonete em barra": (2.50, 12.00),
    "Creme para barbear": (8.00, 25.00),
    "Locao pos-barba": (12.00, 50.00),
    "Protetor solar": (30.00, 80.00),
    "Hidratante corporal": (15.00, 60.00),
    "Lenco umedecido": (6.00, 25.00),
    "Absorvente noturno": (6.00, 22.00),
    "Absorvente diario": (5.00, 18.00),
    "Cotonete": (3.00, 10.00),
    "Fio dental": (5.00, 20.00),
    "Enxaguante bucal": (10.00, 35.00),
    "Pasta de dente clareadora": (10.00, 25.00),
    "Escova de dentes infantil": (5.00, 15.00),
    "Escova de dentes eletrica": (50.00, 200.00),
    "Creme para pentear": (10.00, 35.00),
    "Gel para cabelo": (8.00, 30.00),
    "Mascara capilar": (15.00, 60.00),
    "Condicionador reparador": (12.00, 50.00),
    "Shampoo anticaspa": (15.00, 45.00),
    "Shampoo infantil": (10.00, 35.00),
    "Sabonete intimo": (8.00, 30.00),
    "Creme para assaduras": (10.00, 40.00),
    "Oleo para bebe": (15.00, 50.00),
    "Lenco de papel": (3.00, 15.00),
    "Papel higienico perfumado": (8.00, 30.00),
    "Papel higienico biodegradavel": (10.00, 35.00),
    "Sabonete antibacteriano": (5.00, 20.00),
    "Desinfetante para as maos": (10.00, 30.00),
    "Creme para os pes": (12.00, 40.00),
    "Esmalte de unha": (5.00, 25.00),
    "Removedor de esmalte": (5.00, 20.00),
    "Cera depilatoria": (10.00, 40.00),
    "Lamina de barbear descartavel": (5.00, 25.00),
    "Creme para rugas": (30.00, 100.00),
    "Protetor labial": (10.00, 30.00),
    "Po compacto": (20.00, 80.00),
    "Batom": (15.00, 50.00),
    "Base liquida": (20.00, 90.00),
    "Demaquilante": (15.00, 50.00),
    "Creme para olheiras": (20.00, 80.00),
    "Creme para acne": (15.00, 60.00),
    "Protetor solar facial": (30.00, 100.00),
    "Shampoo seco": (15.00, 45.00),
    "Condicionador sem enxague": (12.00, 40.00),
    "Gel para sobrancelha": (15.00, 50.00),
    "Creme para maos": (10.00, 35.00),
    "Sabonete esfoliante": (8.00, 30.00),
    "Refrigerante": (5.00, 10.00),          # Preço por 2L
    "Suco": (8.00, 15.00),                  # Preço por 1L
    "Agua mineral": (2.00, 5.00),           # Preço por 1,5L
    "Cha": (3.00, 6.00),                    # Preço por caixa com 10 saquinhos
    "Achocolatado": (6.00, 12.00),          # Preço por 400g
    "Cafe": (10.00, 20.00),                 # Preço por 500g
    "Leite": (4.00, 8.00),                  # Preço por 1L
    "Energetico": (6.00, 12.00),            # Preço por lata
    "Cerveja sem alcool": (3.00, 6.00),     # Preço por lata
    "Cerveja zero alcool": (3.00, 6.00),     # Preço por lata
    "Cerveja": (3.00, 6.00),              # Preço por lata ou garrafa
    "Vinho": (20.00, 50.00),              # Preço por garrafa
    "Vodka": (30.00, 80.00),              # Preço por garrafa
    "Whisky": (50.00, 150.00),            # Preço por garrafa
    "Rum": (30.00, 70.00),                # Preço por garrafa
    "Gin": (40.00, 100.00),               # Preço por garrafa
    "Tequila": (50.00, 120.00),           # Preço por garrafa
    "Cachaca": (15.00, 40.00),            # Preço por garrafa
    "Licor": (20.00, 60.00),              # Preço por garrafa
    "Champagne": (50.00, 150.00),         # Preço por garrafa
    "Espumante": (30.00, 80.00),          # Preço por garrafa
    "Sidra": (20.00, 50.00),              # Preço por garrafa
    "Conhaque": (40.00, 100.00),          # Preço por garrafa
    "Vermute": (20.00, 50.00),            # Preço por garrafa
    "Sake": (30.00, 70.00),               # Preço por garrafa
    "Absinto": (50.00, 120.00),           # Preço por garrafa
    "Catuaba": (15.00, 30.00),            # Preço por garrafa
    "Pinga artesanal": (20.00, 50.00),    # Preço por garrafa
    "Vinho tinto": (20.00, 60.00),        # Preço por garrafa
    "Vinho branco": (20.00, 60.00),       # Preço por garrafa
    "Vinho rose": (20.00, 60.00),         # Preço por garrafa
    "Vinho seco": (20.00, 60.00),         # Preço por garrafa
    "Vinho suave": (20.00, 60.00),        # Preço por garrafa
    "Cerveja artesanal": (10.00, 25.00),  # Preço por garrafa
    "Cerveja de trigo": (8.00, 15.00),    # Preço por garrafa
    "Cerveja IPA": (10.00, 20.00),        # Preço por garrafa
    "Cerveja stout": (10.00, 20.00),      # Preço por garrafa
    "Cerveja pilsen": (5.00, 10.00),      # Preço por garrafa
    "Cerveja light": (5.00, 10.00),       # Preço por garrafa
    "Cerveja de milho": (5.00, 10.00),     # Preço por garrafa
    "Leite": (4.00, 8.00),
    "Queijo": (20.00, 80.00),
    "Manteiga": (10.00, 30.00),
    "Margarina": (5.00, 15.00),
    "Iogurte": (3.00, 12.00),
    "Requeijao": (6.00, 15.00),
    "Creme de leite": (3.00, 10.00),
    "Leite condensado": (5.00, 12.00),
    "Chantilly": (8.00, 20.00),
    "Fermento biologico": (2.00, 8.00),
    "Leite em po": (10.00, 40.00),
    "Leite de soja": (8.00, 18.00),
    "Leite de amendoas": (12.00, 25.00),
    "Leite de coco": (5.00, 15.00),
    "Leite de aveia": (10.00, 22.00),
    "Leite sem lactose": (6.00, 12.00),
    "Iogurte grego": (4.00, 12.00),
    "Iogurte natural": (3.00, 10.00),
    "Iogurte com frutas": (4.00, 12.00),
    "Iogurte zero lactose": (5.00, 14.00),
    "Queijo minas": (30.00, 70.00),
    "Queijo prato": (25.00, 60.00),
    "Queijo mussarela": (20.00, 55.00),
    "Queijo parmesao": (50.00, 120.00),
    "Queijo provolone": (40.00, 90.00),
    "Queijo cottage": (10.00, 30.00),
    "Queijo ricota": (8.00, 25.00),
    "Queijo gorgonzola": (50.00, 100.00),
    "Queijo brie": (70.00, 150.00),
    "Queijo coalho": (15.00, 40.00),
    "Yakult": (8.00, 20.00),
    "Danoninho": (6.00, 15.00),
    "Carne bovina": (30.00, 60.00),          # Preço por kg
    "Frango": (15.00, 30.00),                # Preço por kg
    "Peixe": (20.00, 50.00),                 # Preço por kg
    "Salsicha": (10.00, 20.00),              # Preço por kg
    "Presunto": (20.00, 40.00),              # Preço por kg
    "Linguica": (15.00, 30.00),              # Preço por kg
    "Carne moida": (25.00, 50.00),           # Preço por kg
    "Costela": (30.00, 60.00),               # Preço por kg
    "Bacon": (20.00, 40.00),                 # Preço por kg
    "Hamburguer": (15.00, 30.00),            # Preço por kg
    "Presunto cozido": (20.00, 40.00),       # Preço por kg
    "Presunto defumado": (25.00, 50.00),     # Preço por kg
    "Peito de peru": (20.00, 40.00),         # Preço por kg
    "Salame": (25.00, 50.00),                # Preço por kg
    "Mortadela": (15.00, 30.00),             # Preço por kg
    "Rosbife": (30.00, 60.00),               # Preço por kg
    "Carne seca": (40.00, 80.00),            # Preço por kg
    "Charque": (35.00, 70.00),               # Preço por kg
    "Bacalhau": (50.00, 100.00),             # Preço por kg
    "Tilapia": (20.00, 40.00),               # Preço por kg
    "Salmao": (60.00, 120.00),               # Preço por kg
    "Camarao": (50.00, 100.00),              # Preço por kg
    "Lula": (40.00, 80.00),                  # Preço por kg
    "Polvo": (50.00, 100.00),                # Preço por kg
    "Ostra": (30.00, 60.00),                 # Preço por kg
    "Mexilhao": (20.00, 40.00),              # Preço por kg
    "Sushi": (10.00, 20.00),                 # Preço por porção
    "Temaki": (15.00, 30.00),                # Preço por unidade
    "Salsicha hot dog": (10.00, 20.00),      # Preço por kg
    "Salsicha para churrasco": (15.00, 30.00), # Preço por kg
    "Salsicha de frango": (12.00, 25.00),    # Preço por kg
    "Salsicha de peru": (12.00, 25.00),      # Preço por kg
    "Salsicha vegetariana": (15.00, 30.00),  # Preço por kg
    "Hamburguer de frango": (15.00, 30.00),  # Preço por kg
    "Hamburguer vegetariano": (20.00, 40.00), # Preço por kg
    "Hamburguer de soja": (18.00, 35.00),    # Preço por kg
    "Hamburguer de peixe": (20.00, 40.00),   # Preço por kg
    "Hamburguer artesanal": (25.00, 50.00),   # Preço por kg
    "Pizza congelada": (15.00, 30.00),              # Preço por unidade
    "Lasanha congelada": (20.00, 40.00),            # Preço por unidade
    "Pao de queijo congelado": (10.00, 20.00),      # Preço por pacote
    "Hamburguer congelado": (15.00, 30.00),         # Preço por kg
    "Batata frita congelada": (10.00, 20.00),       # Preço por kg
    "Nuggets de frango": (15.00, 30.00),            # Preço por kg
    "Empanado de frango": (20.00, 40.00),           # Preço por kg
    "Empanado de peixe": (25.00, 50.00),            # Preço por kg
    "Escondidinho congelado": (15.00, 30.00),       # Preço por unidade
    "Feijoada congelada": (20.00, 40.00),           # Preço por unidade
    "Risoto congelado": (15.00, 30.00),             # Preço por unidade
    "Yakisoba congelado": (10.00, 20.00),           # Preço por unidade
    "Coxinha congelada": (5.00, 10.00),             # Preço por unidade
    "Kibe congelado": (5.00, 10.00),                # Preço por unidade
    "Pastel congelado": (5.00, 10.00),              # Preço por unidade
    "Torta congelada": (15.00, 30.00),              # Preço por unidade
    "Pao de alho congelado": (10.00, 20.00),        # Preço por pacote
    "Pao de queijo recheado congelado": (12.00, 25.00), # Preço por pacote
    "Sorvete de massa": (10.00, 20.00),             # Preço por pote
    "Sorvete de palito": (2.00, 5.00),              # Preço por unidade
    "Sorvete": (10.00, 20.00),                      # Preço por pote
    "Acai congelado": (15.00, 30.00),               # Preço por kg
    "Polenta congelada": (10.00, 20.00),            # Preço por kg
    "Legumes congelados": (8.00, 15.00),            # Preço por kg
    "Frutas congeladas": (10.00, 20.00),            # Preço por kg
    "Pure de batata congelado": (10.00, 20.00),     # Preço por kg
    "Pao frances congelado": (5.00, 10.00),         # Preço por unidade
    "Pao de forma congelado": (5.00, 10.00),        # Preço por pacote
    "Pao integral congelado": (6.00, 12.00),        # Preço por pacote
    "Pao sirio congelado": (8.00, 15.00),           # Preço por pacote
    "Pao de centeio congelado": (8.00, 15.00),       # Preço por pacote
    "Milho": (3.00, 6.00),              # Preço por lata
    "Ervilha": (3.00, 6.00),            # Preço por lata
    "Seleta de legumes": (4.00, 8.00),  # Preço por lata
    "Sardinha": (5.00, 10.00),          # Preço por lata
    "Atum": (8.00, 15.00),              # Preço por lata
    "Molho de tomate": (2.50, 5.00),    # Preço por lata
    "Extrato de tomate": (3.00, 6.00),  # Preço por lata
    "Palmito": (10.00, 20.00),          # Preço por lata
    "Azeitona": (5.00, 10.00),          # Preço por lata
    "Seleta de legumes": (4.00, 8.00),   # Preço por lata
    "Batata": (3.00, 6.00),
    "Cebola": (2.50, 5.50),
    "Tomate": (4.00, 10.00),
    "Alho": (15.00, 40.00),
    "Cenoura": (2.00, 6.00),
    "Abobora": (3.00, 8.00),
    "Chuchu": (2.00, 5.00),
    "Pimentao": (5.00, 12.00),
    "Alface": (2.00, 5.00),
    "Repolho": (3.00, 7.00),
    "Couve": (2.50, 6.00),
    "Brocolis": (5.00, 15.00),
    "Banana": (3.00, 7.00),
    "Maca": (5.00, 12.00),
    "Laranja": (2.50, 7.00),
    "Limao": (2.00, 6.00),
    "Mamao": (4.00, 10.00),
    "Abacaxi": (5.00, 12.00),
    "Melancia": (6.00, 15.00),
    "Melao": (6.00, 14.00),
    "Uva": (7.00, 20.00),
    "Morango": (8.00, 20.00),
    "Kiwi": (10.00, 25.00),
    "Pera": (6.00, 14.00),
    "Manga": (4.00, 12.00),
    "Goiaba": (3.00, 8.00),
    "Caju": (5.00, 12.00),
    "Maracuja": (6.00, 15.00),
    "Acerola": (7.00, 18.00),
    "Coco": (4.00, 10.00),
    "Pessego": (6.00, 14.00),
    "Ameixa": (8.00, 20.00),
    "Figo": (10.00, 25.00),
    "Caqui": (5.00, 12.00),
    "Jabuticaba": (8.00, 18.00),
    "Cupuacu": (6.00, 14.00),
    "Acai": (10.00, 30.00),
    "Guarana": (6.00, 15.00),
    "Pinhao": (10.00, 25.00),
    "Pepino": (3.00, 6.00),
    "Rabanete": (4.00, 8.00),
    "Beterraba": (3.00, 7.00),
    "Inhame": (5.00, 12.00),
    "Mandioca": (4.00, 10.00),
    "Batata-doce": (3.50, 7.50),
    "Gengibre": (8.00, 18.00),
    "Coentro": (1.50, 4.00),
    "Salsinha": (1.50, 4.00),
    "Pao": (5.00, 10.00),                  # Preço por kg
    "Biscoito": (3.00, 6.00),              # Preço por pacote
    "Bolacha": (3.00, 6.00),               # Preço por pacote
    "Pao de queijo": (10.00, 20.00),       # Preço por kg
    "Pizza": (15.00, 30.00),               # Preço por unidade
    "Lasanha": (20.00, 40.00),             # Preço por unidade
    "Pao integral": (6.00, 12.00),         # Preço por kg
    "Pao frances": (5.00, 10.00),          # Preço por unidade
    "Pao de forma": (5.00, 10.00),         # Preço por pacote
    "Pao sirio": (8.00, 15.00),            # Preço por pacote
    "Pao de centeio": (6.00, 12.00),       # Preço por kg
    "Pao de aveia": (6.00, 12.00),         # Preço por kg
    "Pao de milho": (5.00, 10.00),         # Preço por kg
    "Pao de batata": (5.00, 10.00),        # Preço por kg
    "Pao de queijo congelado": (12.00, 25.00), # Preço por kg
    "Pao de alho": (10.00, 20.00),         # Preço por unidade
    "Biscoito recheado": (4.00, 8.00),     # Preço por pacote
    "Biscoito agua e sal": (3.00, 6.00),   # Preço por pacote
    "Biscoito cream cracker": (3.00, 6.00), # Preço por pacote
    "Biscoito de polvilho": (4.00, 8.00),  # Preço por pacote
    "Biscoito de aveia": (4.00, 8.00),     # Preço por pacote
    "Biscoito de gengibre": (4.00, 8.00),  # Preço por pacote
    "Biscoito de chocolate": (4.00, 8.00), # Preço por pacote
    "Biscoito de coco": (4.00, 8.00),      # Preço por pacote
    "Biscoito de maisena": (4.00, 8.00),   # Preço por pacote
    "Biscoito de nata": (4.00, 8.00),       # Preço por pacote
    "Panela de pressao": (100, 300),
    "Panela antiaderente": (40, 150),
    "Panela de ferro": (60, 250),
    "Panela de barro": (40, 120),
    "Frigideira": (30, 120),
    "Chaleira": (20, 70),
    "Liquidificador": (80, 500),
    "Batedeira": (100, 600),
    "Processador de alimentos": (150, 500),
    "Torradeira": (50, 150),
    "Forno eletrico": (150, 700),
    "Air fryer": (200, 1000),
    "Cafeteira": (30, 250),
    "Jarra eletrica": (50, 200),
    "Faca de cozinha": (20, 200),
    "Tabua de corte": (10, 60),
    "Escorredor de louca": (10, 40),
    "Espatula de silicone": (10, 30),
    "Colher de pau": (5, 20),
    "Ralador de queijo": (10, 50),
    "Descascador de legumes": (10, 30),
    "Assadeira": (20, 80),
    "Forma de bolo": (15, 70),
    "Forma de gelo": (5, 20),
    "Garrafa termica": (20, 150),
    "Caneca": (10, 40),
    "Prato": (5, 30),
    "Tigela": (10, 50),
    "Talheres": (10, 50),
    "Guardanapo de pano": (10, 30),
    "Toalha de mesa": (20, 100),
    "Jogo americano": (10, 50),
    "Vaso decorativo": (20, 150),
    "Porta-temperos": (10, 50),
    "Porta-sabao": (10, 30),
    "Escova de limpeza": (5, 30),
    "Rodo de pia": (10, 40),
    "Vassoura de pia": (10, 50),
    "Organizador de gavetas": (15, 100),
    "Cabide": (5, 20),
    "Caderno": (2, 10),
    "Caneta": (1, 5),
    "Lapis": (1, 5),
    "Borracha": (1, 5),
    "Apontador": (1, 10),
    "Regua": (2, 10),
    "Estojo": (5, 30),
    "Mochila": (50, 300),
    "Cola branca": (3, 10),
    "Tesoura": (5, 30)
}


# Função para gerar valores aleatórios dentro dos intervalos
def gerar_valores(produto, intervalos):
    valores = []
    valores_cust = []
    for item in produto:
        min_valor, max_valor = intervalos[item]
        valor = round(random.uniform(min_valor, max_valor), 2)  # Gera valor aleatório e arredonda para 2 casas decimais
        valor_custo = round(((valor/100)*(100-(random.uniform(15, 35)))),2)
        valores.append(valor)
        valores_cust.append(valor_custo)
        
    return valores, valores_cust

# Gerar lista de valores
valores_unit, valores_custo = gerar_valores(list_apenas_produtos, intervalos_precos)

df_prod_valores = pd.DataFrame({ 
                    "Valor Unit.":  valores_unit, 
                    "Valor Custo": valores_custo, 
                    
})

#df_prod_valores = pd.DataFrame(valores_produto)

df_prod_valores

# Exibir resultados
#print("Produto\t\tValor (R$)")
#for i in range(len(list_apenas_produtos)):
#    print(f"{list_apenas_produtos[i]}\t\t{valores_produto[i]:.2f}")

In [ ]:
import pandas as pd
from datetime import date
from sqlalchemy import create_engine, VARCHAR, DATE, INTEGER, text
import numpy as np

# Configurações de conexão com o SQL Server
server = "DESKTOP-HV85ADA\\PROJETO_MERCADO"
bancodados = "MERCADO_OLTP"
driver = "ODBC Driver 17 for SQL Server"
conexaostring = f"mssql+pyodbc://{server}/{bancodados}?driver={driver}&Trusted_Connection=yes"
engine = create_engine(conexaostring)

#Função para remover espaços desnecessários em todos os tipos de dados
def strip_all_types(col):
    # Verifica se a coluna é do tipo datetime
    if np.issubdtype(col.dtype, np.datetime64):
        # Se for datetime, retorna a coluna original sem modificações
        return col
    
    col_stripped = col.astype(str).str.replace(r'\s+', ' ', regex=True).str.strip() #Converte a coluna para string, remove espaços extras e aplica strip
    
    #Converte de volta para o tipo original da coluna
    if np.issubdtype(col.dtype, np.integer):  # Se for inteiro
        return pd.to_numeric(col_stripped, errors='coerce').astype(col.dtype)
    elif np.issubdtype(col.dtype, np.floating):  # Se for float
        return pd.to_numeric(col_stripped, errors='coerce').astype(col.dtype)
    elif col.name in ["DATA_CRIACAO", "DATA_ATLZ"]:  # Se for uma coluna de data, Converte valores inválidos (como 'None') para NaT (Not a Time)
        return pd.to_datetime(col_stripped, errors='coerce')
    else:  # Se for string ou outro tipo
        return col_stripped

#df_produto = pd.DataFrame({
#    "NOME": ["Hamburguer Seda2"],
#    "ID_CATEGORIA": [11],
#    "PRECO_CUSTO": [11.93],
#    "PRECO_UNIT": [15.17],
#    "ID_UNID_MED": [1],
#    "DATA_CRIACAO": ["2019/12/03"],
#    #"DATA_CRIACAO": pd.to_datetime(["2019-12-03"]).date,
#    "DATA_ATLZ": [None],
#    "VERSAO": [1]
#})
#Tabela e coluna a serem verificadas
tabela = "PRODUTO"
coluna = "NOME"

with engine.connect() as conn:
    # Consulta os valores existentes na coluna "NOME" da tabela "PRODUTO"
    query = text(f"SELECT {coluna} FROM {tabela}")
    lista_valores_existentes = pd.read_sql(query, conn)[coluna].tolist()
    print(lista_valores_existentes)
    print(len(lista_valores_existentes))

    # Filtra os valores que não existem na tabela
    df_produto_filtrado = df_produto[~df_produto["NOME"].isin(lista_valores_existentes)]
    display(df_produto_filtrado)

    # Se houver valores para adicionar
    if not df_produto_filtrado.empty:
        # Limpa os espaços desnecessários
        df_produto_filtrado = df_produto_filtrado.apply(strip_all_types)

        # Define os tipos de dados para a tabela PRODUTO
        tipo_dados_produto = {
            "NOME": VARCHAR(100),
            "ID_CATEGORIA": INTEGER,
            "PRECO_CUSTO": VARCHAR(20),
            "PRECO_UNIT": VARCHAR(20),
            "ID_UNID_MED": INTEGER,
            "DATA_CRIACAO": DATE,
            "DATA_ATLZ": DATE,
            "VERSAO": INTEGER,
        }

        # Exibe os dados que serão adicionados
        print("Dados a serem adicionados:")
        display(df_produto_filtrado)

        # Adiciona os dados ao banco de dados
        df_produto_filtrado.to_sql(
            name=tabela,
            con=engine,
            if_exists="append",
            index=False,
            dtype=tipo_dados_produto
        )

        print("Dados inseridos com sucesso.")
    else:
        print("Nenhum valor adicionado.")

# Fecha a conexão com o banco de dados
engine.dispose()

In [ ]:
#Script para gerar Tabela de Notas Fiscais, Itens de nota e Vendas(OLTP)

import random
import pandas as pd
from datetime import datetime

# Função para gerar data e hora aleatória
def gerar_data_hora(ano_inicio, ano_fim):
    ano = random.randint(ano_inicio, ano_fim)
    mes = random.randint(1, 12)
    if mes in [4, 6, 9, 11]:  # Meses com 30 dias
        dia = random.randint(1, 30)
    elif mes == 2:  # Fevereiro
        dia = random.randint(1, 28)
    else:  # Meses com 31 dias
        dia = random.randint(1, 31)

    hora = random.randint(8, 22)
    minuto = random.randint(0, 59)
    segundo = random.randint(0, 59)

    data_completa = datetime(ano, mes, dia, hora, minuto, segundo)
    return data_completa

# Função para gerar a nota fiscal
def gerar_nota_fiscal(numero_nota, lista_produtos, quant_maxima_itens, data_nota):
    # Garantir que a quantidade de itens únicos seja pelo menos 1
    quant_item_unicos = random.randint(1, quant_maxima_itens)  # Gerar quantidade aleatória de itens únicos na nota fiscal
    #print(f"Quantidade de itens únicos na nota {numero_nota}: {quant_item_unicos}")

    lista_saida = []  # Lista para armazenar os itens da nota fiscal
    lista_NOTAFISCAL = []

    id_forma_pgmt = random.randint(1, 8)
    total_clientes_db = (engine.connect().execute(text("SELECT COUNT(IDCLIENTE) FROM [MERCADO_OLTP].[dbo].CLIENTE"))).scalar()
    id_cliente = random.randint(1, total_clientes_db)
    total_lojas_db = (engine.connect().execute(text("SELECT COUNT(IDLOJA) FROM [MERCADO_OLTP].[dbo].LOJA"))).scalar()
    id_loja = random.randint(1, total_lojas_db)
    id_cfop = random.randint(1, 6)
    chaveacesso = ''.join([str(random.randint(0, 9)) for _ in range(44)])
    data_criacao = data_nota.date()
    data_atlz = ""
    versao = 1
    
    # Gerar os itens da nota fiscal
    for _ in range(quant_item_unicos):
        produto = random.choice(lista_produtos)
        if produto in df_prod_valores["NOME"].values:
            preco_unit = df_prod_valores.loc[df_prod_valores["NOME"] == produto, "PRECO_UNIT"].values[0]
            preco_unit = round(float(preco_unit), 2)  # Preço unitário com 2 casas decimais
            quantidade = random.randint(1, 10)  # Quantidade aleatória do produto
            preco_total_item = round(quantidade * preco_unit, 2)  # Total do item
            id_produto = listaprodutosPROVISORIO.index(produto)+1
            lista_saida.append((data_nota, numero_nota, produto, quantidade, preco_unit, preco_total_item, 
                                id_produto, id_forma_pgmt, id_cliente, id_loja, id_cfop, chaveacesso, data_criacao, data_atlz, versao))

    # Calcular o total da nota fiscal
    preco_total_nota = round(sum(item[5] for item in lista_saida), 2)
    
    # Adicionar o total da nota a cada item da lista
    lista_saida = [item + (preco_total_nota,) for item in lista_saida]
   
    lista_NOTAFISCAL.append((numero_nota, data_nota, id_forma_pgmt, id_cliente, id_loja, id_cfop, chaveacesso, data_criacao, 
                             data_atlz, versao,preco_total_nota))
    

    # Criar um DataFrame com os dados da nota fiscal
    df_nota_fiscal = pd.DataFrame(
        lista_saida, columns=["DATA_NOTAFISCAL", "Numero_Nota", "Produto", "QUANTIDADE", "PRECO_UNIT", "TOTAL_ITEM", 
                              "ID_PRODUTO","ID_FORMA_PGMT", "ID_CLIENTE", "ID_LOJA", "ID_CFOP","CHAVE_ACESSO","DATA_CRIACAO",
                              "DATA_ATLZ","VERSAO","TOTAL_NOTA"]
    )

    df_2 = pd.DataFrame(
        lista_NOTAFISCAL, columns=["Numero_Nota", "DATA_NOTAFISCAL", "ID_FORMA_PGMT", "ID_CLIENTE", "ID_LOJA", "ID_CFOP", 
                                   "CHAVE_ACESSO", "DATA_CRIACAO", "DATA_ATLZ", "VERSAO","TOTAL_NOTA"]
    )

    return df_nota_fiscal, df_2

# Quantidade de notas fiscais
numero_inicial_nota = 1000
numero_maximo_nota = 1003
quant_maxima_itens = 10

# Lista para armazenar todas as notas fiscais
df_notas_fiscais = []
df_notas_fiscais_2 = []

# Gerar notas fiscais para o intervalo definido
for numero_nota in range(numero_inicial_nota, numero_maximo_nota + 1):
    data_nota = gerar_data_hora(2024, 2024)
    df_nota_fiscal, df_2 = gerar_nota_fiscal(numero_nota, listaprodutosPROVISORIO, quant_maxima_itens, data_nota)
    df_notas_fiscais.append(df_nota_fiscal)
    df_notas_fiscais_2.append(df_2)


# Concatenar todas as notas fiscais em um único DataFrame
df_todas_notas = pd.concat(df_notas_fiscais, ignore_index=True)
df_todas_notas2 = pd.concat(df_notas_fiscais_2, ignore_index=True)


df_ITEMNOTA = pd.DataFrame({
                'ID_NOTAFISCAL':  df_todas_notas['Numero_Nota'],
                'ID_PRODUTO':     df_todas_notas['ID_PRODUTO'],
                'QUANTIDADE':     df_todas_notas['QUANTIDADE'],
                'PRECO_UNIT':     df_todas_notas['PRECO_UNIT'],
                'TOTAL_ITEM':     df_todas_notas['TOTAL_ITEM'],
                'DATA_CRIACAO':   df_todas_notas['DATA_CRIACAO'],
                'DATA_ATLZ':      df_todas_notas['DATA_ATLZ'],
                'VERSAO':         df_todas_notas['VERSAO'],
})

df_NOTAFISCAL = pd.DataFrame({
                'DATA':           df_todas_notas2['DATA_NOTAFISCAL'],
                'TOTAL_NOTA':     df_todas_notas2['TOTAL_NOTA'],
                'ID_FORMA_PGMT':  df_todas_notas2['ID_FORMA_PGMT'],
                'ID_CLIENTE':     df_todas_notas2['ID_CLIENTE'],
                'ID_LOJA':        df_todas_notas2['ID_LOJA'],
                'ID_CFOP':        df_todas_notas2['ID_CFOP'],
                'CHAVE_ACESSO':   df_todas_notas2['CHAVE_ACESSO'],
                'DATA_CRIACAO':   df_todas_notas2['DATA_CRIACAO'],
                'DATA_ATLZ':      df_todas_notas2['DATA_ATLZ'],
                'VERSAO':         df_todas_notas2['VERSAO'],
})

df_VENDA = pd.DataFrame({
                'DATA':           df_todas_notas2['DATA_NOTAFISCAL'],
                'ID_NOTAFISCAL':  df_todas_notas2['Numero_Nota'],
                'ID_CLIENTE':     df_todas_notas2['ID_CLIENTE'],
                'ID_LOJA':        df_todas_notas2['ID_LOJA'],
                'ID_FORMA_PGMT':  df_todas_notas2['ID_FORMA_PGMT'],
                'TOTAL_VENDA':    df_todas_notas2['TOTAL_NOTA'],
                'DATA_CRIACAO':   df_todas_notas2['DATA_CRIACAO'],
                'DATA_ATLZ':      df_todas_notas2['DATA_ATLZ'],
                'VERSAO':         df_todas_notas2['VERSAO'],
})

#Exibir todas as notas fiscais
#print("\nTodas as Notas Fiscais:")
#print(df_todas_notas)
display(df_todas_notas)
display(df_ITEMNOTA)
display(df_NOTAFISCAL)
display(df_VENDA)